# Notebook Objective and Setup

BGG03 is the scrubbing and cleaning of the various data obtained in notebooks BGG01 and BGG02. The following datasets are cleaned, constructed, or otherwise prepared for EDA and modeling.

    * Games
    * Mechanics
    * Subcategories
    * Designers
    * Artists
    * Publishers
    * Awards
    * Ratings Distribution
    * Comments
    * Ratings Matrix

## Package Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import regex as re
import time
import os
import gc
import json

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

# NLP tools
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

## Notebook Functions

In [2]:
def integer_reduce(data, columns, fill_value=0):
    '''
    Reduces an integer type to its smallest memory size type
    
    Inputs:
    data: dataframe to reduce
    columns: columns to reduce
    fill_value: fill value to use if none
    
    Returns: 
    data: dataframe with memory reduced data types
    '''
    for column in columns:
        print(column)
        data[column].fillna(fill_value, inplace=True)
        if (data[column].max() <= 127) & (data[column].min() >= -128):
            data[column] = data[column].astype('int8')
        elif (data[column].max() <= 32767) & (data[column].min() >= -32768):
            data[column] = data[column].astype('int16')
        elif (data[column].max() <= 2147483647) & (data[column].min() >= -2147483648):
            data[column] = data[column].astype('int32')
        
    return data

In [3]:
def text_block_processor(text):
    '''Takes a block of text. Divides block into sentences with words lemmatized.
    Sends each sentence to word processor. Concatenates all words into one string
    Otherwise returns string of cleaned and processed words from text block
    
    ARGUMENTS:
    block of text
    '''
    
    text = str(text)
    line = re.sub(r'[^a-zA-Z\s]', '', text).lower() # removes all special characters and numbers, and makes lower case
    line2 = re.sub(r'\s{2}', '', line).lower() # removes extra blocks of 2 spaces
    tokens = nlp(line)
    words = []
    for token in tokens:
        if token.is_stop == False:
            token_preprocessed = token.lemma_
            if token_preprocessed != '': # only continues if returned word is not empty
                words.append(token_preprocessed) # appends word to list of words
    line = ' '.join(words)
    
    return line


In [4]:
def fix_numbers(x):
    '''
    Checks for numbers or strings
    If a string, strips off the "k" and multiply by 10000
    Sends back cleaned int
    '''
    
    if type(x) is int:
        return int(x)
    
    if str.endswith(x, 'k'):
        x = str(x).strip('k')
        new_num = int(float(x)*1000)
        return int(new_num)
    
    else: 
        return int(x)

In [5]:
def clean_ratings(id_num, game_ids):
    '''
    Loads and cleans a raw user ratings file
    Drops game ids not present in games file
    Drops users with fewer than 10 ratings
    
    Inputs:
    id_num: the appendation of the file to find the path
    game_ids: list of game ids in the games file
    
    Outputs:
    Cleaned user ratings file
    '''
    
    print('\nCleaning Frame #'+str(id_num))
    
    #load in raw users file according to id_num inputted
    path = 'userid/user_ratings'+str(id_num)+'.pkl'
    users = pd.read_pickle(path)
    
    # convert all datatypes to float
    float_converted = users.astype('float')
    
    # delete and clean up raw users file
    del users
    gc.collect()
    
    # create intersection between user file and game list ids
    float_converted.columns = float_converted.columns.astype('int32')
    cleaned = float_converted[float_converted.columns.intersection(game_ids)]
    
    # delete and clean up
    del float_converted
    gc.collect()
    
    # make a list of users with fewer than 5 user ratings
    sums = cleaned.count(axis=1)<5
    # get indices for the rows with fewer than 5 ratings
    drop_these = sums.loc[sums==True].index
    # drop the users with fewer than 5 ratings
    cleaned.drop(drop_these, axis=0, inplace=True)
    
    
    # print memory usage
    print(cleaned.info())
    
    # return cleaned file
    return cleaned

In [6]:
def create_ratings_file(start_file, end_file, game_ids):
    '''
    Puts together dataframes from a range of files
    Each file calls the clean_ratings function
    Then all files in range are concatenated
    
    Inputs:
    start_file: start of file name appendation
    end_file: end file name appendation
    game_ids_list: list of game ids in the games file
    
    Outputs:
    Cleaned and concatenated master file
    
    '''
    
    # make an empty dataframe
    master_file = pd.DataFrame()
    
    # for each number in the range from start to end:
    for id_num in np.arange(start_file,end_file+1,1):
        print(id_num)
        # clean the file calling clean_ratings
        cleaned_item = clean_ratings(id_num, game_ids)
        # append the file to the dataframe
        master_file = pd.concat([master_file, cleaned_item], axis=0)
    
    master_file.drop_duplicates(keep='first', inplace=True)
    
    # clean up
    del cleaned_item
    gc.collect()
    
    return master_file

In [7]:
def process_dataframe_ratings(x, user_ratings, raw_ratings):
    
    try:
        user_ratings[x['Username']][x['BGGId']] = float(x['Rating']) 
    
    except:
        user_ratings[x['Username']] = {}
        user_ratings[x['Username']][x['BGGId']] = float(x['Rating']) 
    
    raw_ratings[x['BGGId']].append(x['Rating'])

# Games Files

In [8]:
games = pd.read_pickle('data_dirty/games.pkl')
games

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumImplementations,IsReimplementation,Family,Theme,Category,Kickstarted,ImagePath,NumAwards,NumFans,NumPageViews,RulesPosts,TotalPosts,IsExpansion,Rank:boardgame,Rank:strategygames,Rank:abstracts,Rank:familygames,Rank:thematic,Rank:cgs,Mechanism,Rank:wargames,Setting,Rank:partygames,Rank:childrensgames,Rank:boardgameaccessory,Rank:videogame,Rank:amiga,Rank:commodore64,Rank:arcade,Rank:atarist,Rank:rpgitem
0,1,Die Macher,Die Macher is a game about seven sequential po...,1986,4.3206,7.61428,7.10363,1.579790,3,5,14.366667,1.395833,5,"[4, 5]",7498,501,2039,761,240,240,240,14,5354,NaN,2,0,0,0,Classic Line (Valley Games),NaN,NaN,NaN,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,NaN,NaN,NaN,NaN,NaN,NaN,316.0,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Dragonmaster,Dragonmaster is a trick-taking card game based...,1981,1.9630,6.64537,5.78447,1.454400,3,4,NaN,27.000000,None,[],1285,72,191,54,30,30,30,12,562,NaN,0,0,2,1,NaN,NaN,NaN,NaN,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,NaN,NaN,NaN,NaN,NaN,NaN,3993.0,1577.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Samurai,Samurai is set in medieval Japan. Players comp...,1998,2.4859,7.45601,7.23994,1.182270,2,4,9.307692,1.000000,3,"[2, 3, 4]",15578,799,3450,1451,60,30,60,10,15146,NaN,6,0,1,0,Euro Classics (Reiner Knizia),Samurai,NaN,NaN,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,NaN,NaN,NaN,NaN,NaN,NaN,224.0,166.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Tal der Könige,When you see the triangular box and the luxuri...,1992,2.6667,6.60006,5.67954,1.231290,2,4,13.000000,256.000000,None,[],638,54,123,30,60,60,60,12,340,NaN,0,0,0,0,NaN,NaN,NaN,NaN,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,NaN,NaN,NaN,NaN,NaN,NaN,5345.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Acquire,"In Acquire, each player strategically invests ...",1964,2.5031,7.33861,7.14189,1.335830,2,6,11.410256,21.152941,4,"[3, 4, 5]",23735,548,2671,1606,90,90,90,12,18655,NaN,6,2,0,0,3M Bookshelf,NaN,NaN,NaN,https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...,NaN,NaN,NaN,NaN,NaN,NaN,290.0,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,Salvage,An oil tanker is on fire and you are a rescue ...,2021,0.0000,7.45000,5.52462,1.554930,3,4,12.000000,NaN,None,[],46,10,34,0,40,20,40,10,34,NaN,0,0,0,0,NaN,NaN,NaN,NaN,https://cf.geekdo-images.com/Uzx3n6dd0LGC2sGyS...,NaN,NaN,NaN,NaN,NaN,NaN,13730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21921,347521,Blitzkrieg!: World War Two in 20 Minutes,New Square Edition: Now includes Nippon expans...,2019,1.6667,7.92276,5.56323,0.894204,1,2,16.000000,98.000000,None,[],152,16,52,6,45,20,45,14,45,NaN,0,0,1,1,NaN,NaN,NaN,NaN,https://cf.geekdo-images.com/eG4UBkwA9SNNEFUkE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21922,348955,Rock Paper Scissors: Deluxe Edition,"For millions of years, people have been forced...",2021,4.5625,6.96970,5.53964,4.052570,1,3,12.000000,155.000000,None,[],6,2,13,96,1,1,1,12,66,NaN,0,0,0,0,NaN,NaN,NaN,1,https://cf.geekdo-images.com/TRCPgnDKCu8civrbB...,NaN,NaN,NaN,NaN,NaN,NaN,11507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21923,349131,Splitter,"In Splitter, you must group numbers together t...",2021,1.0000,6.48235,5.52631,1.160070,1,12,NaN,31.000000,None,[],110,10,49,2,15,15,15,8,51,NaN,0,0,0

In [9]:
games['Theme'].unique()

array([nan, 'Samurai', 'Robots', 'Weather', 'Nuclear option',
       'Cthulhu Mythos', 'Psychic Powers', 'Food / Cooking',
       'Archaeology / Paleontology', 'Deserts', 'Oil / Gas / Petroleu',
       'Alternate History', 'Trucks', 'Art', 'Bacteria',
       'Cannibals / Cannibalis', 'Safaris', 'Video Game Theme: Nintendo',
       'Gardening', 'Construction', 'Ninjas',
       'Native Americans / First Peoples',
       'King Arthur / The Knights of the Round Table / Camelot',
       'Tropical', 'Gladiators', 'Colonial', 'Mystery / Cri', 'Vikings',
       'Kaiju', 'Time Travel', 'Circus', 'Mail / Stamps / The Post Offic',
       'Druids', 'Anime / Manga', 'Steampunk', 'Mayans', 'Polic',
       'Hot Air Balloons', 'Anthropomorphic Animals', 'Under the Sea',
       'Templ', 'Mining', 'Attorneys / Courts', 'Aboriginal Australians',
       'Animal Battles', 'Jewelry', 'Mad Science / Mad Scientist',
       'Retro', "Rubik's Cub", 'Climate Chang', 'Fantasy Sports',
       'Evolution', 'Superhe

In [10]:
# Get info, make note of datatypes and memory usage
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    21925 non-null  object 
 1   Name                     21925 non-null  object 
 2   Description              21925 non-null  object 
 3   YearPublished            21925 non-null  object 
 4   GameWeight               21925 non-null  float64
 5   AvgRating                21925 non-null  float64
 6   BayesAvgRating           21925 non-null  float64
 7   StdDev                   21925 non-null  float64
 8   MinPlayers               21925 non-null  object 
 9   MaxPlayers               21925 non-null  object 
 10  ComAgeRec                16395 non-null  float64
 11  LanguageEase             16034 non-null  float64
 12  BestPlayers              1981 non-null   object 
 13  GoodPlayers              21925 non-null  object 
 14  NumOwned              

In [11]:
#drops = games.loc[games['BGGId']==0].index # get indices of any games with no BGGId
#drops
#games.drop(games.loc[drops].index, axis=0, inplace=True) # drop games with no BGGId

In [12]:
games['Mechanism'].unique()

array([nan, '4X', 'Bluffing – Dice Cup', 'Judging G', 'Card L',
       'Connectio', 'Turnless Trading', 'Finger Flicking G',
       'Give a Clue / Get a Clu', 'Hunt', "Deduction – Blind Man's Bluff",
       'Tableau Building', 'Hidden Movement', 'Modelling/Shaping',
       'Campaign G', 'Story Creatio', 'Roll-and-Writ', 'Drawing',
       'Tower Stacking', 'Artificial Player', 'Maxit',
       'Partially overlapping card', 'Dice Drafting',
       'Image Interpretatio', 'Legacy', 'Flip-and-Writ'], dtype=object)

In [13]:
games['Setting'].unique()

array([nan, 'The Realms of Terrinoth', 'Das Schwarze Au',
       'Star Fleet Univers', 'Warhammer Fantasy Board Games',
       'Warhammer Fantasy Wargames',
       'The Weird West (Pinnacle Entertainment)', 'Dragonlanc',
       'Mage Knigh', 'Warhammer Fantasy Card Games', 'Dungeons & Dragons',
       '2000 AD (Warlord Games)', 'Warhammer 40,000 Board Games',
       'Mutant Chronicles', 'Glorantha', 'Traveller Univers',
       'Legend of the Five Rings', 'World of Darkness (White Wolf)',
       'Shadowru', 'Warhammer 40,000 Wargames', 'Renegade Legio',
       '1889 univers', 'Star Trek - The Kobayashi Maru',
       'Warhammer 40,000 Card Games', 'Ophidian (Fleer/Skybox)',
       'Warcraf', 'Void (Viridian Solar System)', 'Zavandor',
       "Corvus Belli's Infinity Univers", 'Neuroshima', 'Anima Univers',
       'Bodgers', 'World of Zanziar', 'Android',
       'Dystopian Universe (Indie Boards & Cards)', 'MERC', 'Onivers',
       'World of Indines', 'Rise of the Occulites Univers', 'Kil

In [14]:
games.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicate entires

# drop non-boardgame related information
games.drop(['NumAwards', 'NumFans', 'NumPageViews', 'RulesPosts', 'TotalPosts', 'Category', 'IsExpansion', 'Rank:rpgitem', 'Rank:boardgameaccessory', 'Rank:videogame', 'Rank:amiga', 'Rank:commodore64', 'Rank:arcade', 'Rank:atarist', 'Setting', 'Mechanism'], axis=1, inplace=True)

games.shape # check shape of file

(21925, 41)

In [15]:
# what are the columns?
games.columns

Index(['BGGId', 'Name', 'Description', 'YearPublished', 'GameWeight',
       'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
       'ComAgeRec', 'LanguageEase', 'BestPlayers', 'GoodPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Family', 'Theme', 'Kickstarted', 'ImagePath',
       'Rank:boardgame', 'Rank:strategygames', 'Rank:abstracts',
       'Rank:familygames', 'Rank:thematic', 'Rank:cgs', 'Rank:wargames',
       'Rank:partygames', 'Rank:childrensgames'],
      dtype='object')

In [16]:
games['BestPlayers'].fillna(0, inplace=True) # Fill NaN on BestPlayers
games['BestPlayers'].unique() # What are the unique BestPlayers entries?

array(['5', 0, '3', '4', '6', '2', '7', '8', '1', '13', '0+', '12', '15',
       '14', '9', '3+'], dtype=object)

In [17]:
# Clean up Best Players so all are integers
games.loc[(games['BestPlayers']=='3+'), 'BestPlayers'] = 3
games.loc[(games['BestPlayers']=='0+'), 'BestPlayers'] = 4

# change dtype on BestPlayers to int8
games['BestPlayers'] = games['BestPlayers'].astype('int8')

In [18]:
# Add Categories with binary flags
games.loc[games['Rank:thematic'].notna(), 'Cat:Thematic'] = int(1)
games.loc[games['Rank:strategygames'].notna(), 'Cat:Strategy'] = int(1)
games.loc[games['Rank:wargames'].notna(), 'Cat:War'] = int(1)
games.loc[games['Rank:familygames'].notna(), 'Cat:Family'] = int(1)
games.loc[games['Rank:cgs'].notna(), 'Cat:CGS'] = int(1)
games.loc[games['Rank:abstracts'].notna(), 'Cat:Abstract'] = int(1)
games.loc[games['Rank:partygames'].notna(), 'Cat:Party'] = int(1)
games.loc[games['Rank:childrensgames'].notna(), 'Cat:Childrens'] = int(1)

In [19]:
# prepare different column sets for memory integer reduction

# integer reduction with fill_values of 0
int_columns=['BGGId', 'YearPublished', 'MinPlayers', 'MaxPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime','MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Kickstarted', 'Cat:Thematic', 'Cat:Strategy', 
        'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party', 'Cat:Childrens']

# integer reduction with fill_values of 21926 (lower is better on these)
ranks = ['Rank:boardgame', 'Rank:thematic', 'Rank:strategygames',
       'Rank:wargames', 'Rank:familygames', 'Rank:cgs', 'Rank:abstracts',
       'Rank:partygames', 'Rank:childrensgames']

# call integer_reduce on the sets
games = integer_reduce(games, int_columns, fill_value=0)

games = integer_reduce(games, ranks, fill_value=21926)

games.info() # recheck data types and memory usage

BGGId
YearPublished
MinPlayers
MaxPlayers
NumOwned
NumWant
NumWish
NumWeightVotes
MfgPlaytime
ComMinPlaytime
ComMaxPlaytime
MfgAgeRec
NumUserRatings
NumComments
NumAlternates
NumExpansions
NumImplementations
IsReimplementation
Kickstarted
Cat:Thematic
Cat:Strategy
Cat:War
Cat:Family
Cat:CGS
Cat:Abstract
Cat:Party
Cat:Childrens
Rank:boardgame
Rank:thematic
Rank:strategygames
Rank:wargames
Rank:familygames
Rank:cgs
Rank:abstracts
Rank:partygames
Rank:childrensgames
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21925 entries, 0 to 21924
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BGGId                21925 non-null  int32  
 1   Name                 21925 non-null  object 
 2   Description          21925 non-null  object 
 3   YearPublished        21925 non-null  int16  
 4   GameWeight           21925 non-null  float64
 5   AvgRating            21925 non-null  float64
 6   BayesAvgRating    

In [20]:
# any games with no name listed?
games.loc[games['Name']==0]

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,GoodPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumImplementations,IsReimplementation,Family,Theme,Kickstarted,ImagePath,Rank:boardgame,Rank:strategygames,Rank:abstracts,Rank:familygames,Rank:thematic,Rank:cgs,Rank:wargames,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens


In [21]:
# if so, drop them and reset index
#empty_games = list(games.loc[games['Name']==0].index)
#games.drop(games.index[empty_games], inplace=True)
#games.reset_index(inplace=True, drop=True)

In [22]:
# Drop all games that are not yet released, then reset index
#not_released = list(games.loc[games['YearPublished']>2021].index)
#games.drop(games.index[not_released], inplace=True)
#games.reset_index(inplace=True, drop=True)

In [23]:
themes = pd.DataFrame(games['Theme'])
games.drop('Theme', axis=1, inplace=True)

In [24]:
# process the Description column text
games['Description'] = games['Description'].apply(lambda x: text_block_processor(x))

In [25]:
games.reset_index(inplace=True, drop=True)

In [26]:
# save to file
games.to_csv('data_kaggle/games.csv', index=False)

In [27]:
# All games with over 12 players are set at 13 players.
games.loc[games['MaxPlayers']>12, 'MaxPlayers'] = 13

In [28]:
# Games with min players of 0, we will set their min players = 2
games.loc[games['MinPlayers']<1, 'MaxPlayers'] = 2

In [29]:
# save to file
games.to_pickle('data_cleaned/games.pkl')

## Game ID Lookup

In [30]:
# dictionary of game IDs-Names

# Load games
games = pd.read_pickle('data_cleaned/games.pkl')

# lists of game ids and game names
game_ids = list(games['BGGId'])
game_names = list(games['Name'])

# make lookup dictionary
game_id_lookup = {}

# store ids and names in lookup dictionary
for key, item in zip(game_ids, game_names):
    game_id_lookup[key] = item

    
#del games
#gc.collect()

game_id_lookup

{1: 'Die Macher',
 2: 'Dragonmaster',
 3: 'Samurai',
 4: 'Tal der Könige',
 5: 'Acquire',
 6: 'Mare Mediterraneum',
 7: 'Cathedral',
 8: 'Lords of Creation',
 9: 'El Caballero',
 10: 'Elfenland',
 11: 'Bohnanza',
 12: 'Ra',
 13: 'Catan',
 14: 'Basari',
 15: 'Cosmic Encounter',
 16: 'MarraCash',
 17: 'Button Men',
 18: 'RoboRally',
 19: 'Wacky Wacky West',
 20: 'Full Metal Planète',
 21: 'Gateway to the Stars',
 22: 'Magic Realm',
 23: 'Divine Right',
 24: 'Twilight Imperium',
 25: 'Battlemist',
 26: 'Age of Renaissance',
 27: 'Supremacy: The Game of the Superpowers',
 28: 'Illuminati',
 29: 'Terrain Vague',
 30: 'Dark Tower',
 31: 'Dark World',
 32: 'Buffalo Chess',
 34: 'Arkham Horror',
 36: 'Federation & Empire',
 37: 'Dragon Masters',
 38: 'Runes',
 39: 'Darkover',
 40: 'Borderlands',
 41: "Can't Stop",
 42: 'Tigris & Euphrates',
 43: 'Airlines',
 44: 'David & Goliath',
 45: 'Perudo',
 46: 'Medici',
 47: 'Chinatown',
 48: 'Krieg und Frieden',
 49: 'Mamma Mia!',
 50: 'Lost Cities',
 

In [25]:
# save dictionary
with open('data_cleaned/game_id_lookup.json', 'w') as convert_file:
    convert_file.write(json.dumps(game_id_lookup))

## Mechanics and Subcategories

### Load and Clean Mechanics

In [31]:
# Load mechanics and check memory usage
mechanics = pd.read_pickle('data_dirty/mechanics.pkl')
mechanics

,BGGId,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,Set Collection,Tile Placement,Action Points,Investment,Market,Square Grid,Stock Holding,Victory Points as a Resource,Enclosure,Pattern Building,Pattern Recognition,Modular Board,Card Drafting,Network and Route Building,Point to Point Movement,Melding and Splaying,Negotiation,Trading,Auction: Once Around,Closed Economy Auction,Constrained Bidding,Push Your Luck,Income,Race,Random Production,Variable Set-up,Roll / Spin and Move,Variable Player Powers,Action Queue,Bias,Grid Movement,Lose a Turn,Programmed Movement,Scenario / Mission / Campaign Game,Voting,Events,Paper-and-Pencil,Player Elimination,Role Playing,Movement Points,Simulation,Variable Phase Order,Area Movement,Commodity Speculation,Cooperative Game,Deduction,Rock-Paper-Scissors,Sudden Death Ending,Connections,Highest-Lowest Scoring,Betting and Bluffing,Multiple-Lot Auction,Memory,Score-and-Reset Game,Layering,Map Addition,Secret Unit Deployment,Increase Value of Unchosen Resources,Ratio / Combat Results Table,Turn Order: Progressive,Take That,Team-Based Game,Campaign / Battle Card Driven,Tech Trees / Tech Tracks,Turn Order: Stat-Based,Auction: Dutch,Player Judge,Chit-Pull System,Three Dimensional Movement,Action Drafting,Minimap Resolution,Stat Check Resolution,Action Timer,Pick-up and Deliver,Auction: English,Auction: Sealed Bid,Auction: Turn Order Until Pass,Map Deformation,Bingo,Crayon Rail System,Multiple Maps,Hot Potato,Hidden Roles,Flicking,Line Drawing,Tug of War,Pattern Movement,Static Capture,Selection Order Bid,Different Dice Movement,Chaining,Ladder Climbing,Predictive Bid,Solo / Solitaire Game,Line of Sight,Critical Hits and Failures,Interrupts,Zone of Control,Bribery,End Game Bonuses,Area-Impulse,Worker Placement,Measurement Movement,Map Reduction,Real-Time,Resource to Move,Mancala,Ownership,Kill Steal,Hidden Movement,Track Movement,Deck Construction,Drafting,TableauBuilding,Turn Order: Role Order,Turn Order: Auction,Prisoner's Dilemma,Hidden Victory Points,Movement Template,Slide/Push,Auction: Dexterity,Targeted Clues,Command Cards,Turn Order: Claim Action,Grid Coverage,Relative Movement,Action/Event,Card Play Conflict Resolution,"I Cut, You Choose",Die Icon Resolution,Turn Order: Random,Elapsed Real Time Ending,Advantage Token,"Worker Placement, Different Worker Types",Storytelling,Catch the Leader,Acting,Singing,Stacking and Balancing,Roles with Asymmetric Information,Traitor Game,Moving Multiple Units,Semi-Cooperative Game,Communication Limits,Time Track,Speed Matching,Cube Tower,Re-rolling and Locking,Impulse Movement,Loans,Delayed Purchase,"Deck, Bag, and Pool Building",Move Through Deck,Single Loser Game,Matching,Induction,Physical Removal,Narrative Choice / Paragraph,Pieces as Map,Follow,Finale Ending,Auction: Fixed Placement,Order Counters,Contracts,Passed Action Token,King of the Hill,Action Retrieval,Turn Order: Pass Order,Force Commitment,Rondel,Worker Placement with Dice Workers,Automatic Resource Growth,Auction: Dutch Priority,Legacy,Legacy Game
0,1,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [32]:
mechanics.drop_duplicates(subset='BGGId', keep='first', inplace=True)  # drop duplicates
mechanics.fillna(0, inplace=True) # fill nan

mechanics.reset_index(inplace=True, drop=True) # reset index
mechanics.info() # check memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Columns: 185 entries, BGGId to Legacy Game
dtypes: float64(184), int64(1)
memory usage: 30.9 MB


In [33]:
# Clean up mechanics
# Here we are using our domain knowledge to compact several different catogories into one

auction_list = ['Auction: Dexterity','Auction: Dutch','Auction: Dutch Priority',
                'Auction: Fixed Placement','Auction: English','Auction: Once Around','Auction: Sealed Bid',
                'Auction: Turn Order Until Pass','Multiple-Lot Auction','Closed Economy Auction','Selection Order Bid',
                'Constrained Bidding']

turn_order_list = ['Turn Order: Auction','Turn Order: Claim Action','Turn Order: Pass Order',
                   'Turn Order: Progressive','Turn Order: Random','Turn Order: Role Order','Turn Order: Stat-Based']

dumb_physical_list = ['Acting','Hot Potato','Singing','Rock-Paper-Scissors']

drafting = ['Card Drafting']

legacy = ['Legacy']

worker_placement = ['Worker Placement with Dice Workers','Worker Placement, Different Worker Types'] #'Worker Placement',

dexterity = ['Stacking and Balancing', 'Flicking']

# compacting categories here:

for item in worker_placement:
    mechanics.loc[mechanics[item]==1, 'Worker Placement'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)

for item in auction_list:
    mechanics.loc[mechanics[item]==1, 'Auction/Bidding'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)

mechanics['Dexterity'] = int(0)
for item in dexterity:
    mechanics.loc[mechanics[item]==1, 'Dexterity'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)
    
mechanics['Physical'] = int(0)
for item in dumb_physical_list:
    mechanics.loc[mechanics[item]==1, 'Physical'] = int(1)
    mechanics.drop([item], axis=1, inplace=True)
    
mechanics.loc[mechanics['Card Drafting']==1, 'Drafting'] = int(1)

mechanics.loc[mechanics['Legacy']==1, 'Legacy Game'] = int(1)

mechanics.drop(turn_order_list, axis=1, inplace=True)
mechanics.drop(['Card Drafting','Legacy'], axis=1, inplace=True)

In [34]:
columns = mechanics.columns

# call integer_reduce on the sets
mechanics = integer_reduce(mechanics, columns, fill_value=0)

BGGId
Alliances
Area Majority / Influence
Auction/Bidding
Dice Rolling
Hand Management
Simultaneous Action Selection
Trick-taking
Hexagon Grid
Once-Per-Game Abilities
Set Collection
Tile Placement
Action Points
Investment
Market
Square Grid
Stock Holding
Victory Points as a Resource
Enclosure
Pattern Building
Pattern Recognition
Modular Board
Network and Route Building
Point to Point Movement
Melding and Splaying
Negotiation
Trading
Push Your Luck
Income
Race
Random Production
Variable Set-up
Roll / Spin and Move
Variable Player Powers
Action Queue
Bias
Grid Movement
Lose a Turn
Programmed Movement
Scenario / Mission / Campaign Game
Voting
Events
Paper-and-Pencil
Player Elimination
Role Playing
Movement Points
Simulation
Variable Phase Order
Area Movement
Commodity Speculation
Cooperative Game
Deduction
Sudden Death Ending
Connections
Highest-Lowest Scoring
Betting and Bluffing
Memory
Score-and-Reset Game
Layering
Map Addition
Secret Unit Deployment
Increase Value of Unchosen Resources

In [35]:
mechanics

,BGGId,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,Set Collection,Tile Placement,Action Points,Investment,Market,Square Grid,Stock Holding,Victory Points as a Resource,Enclosure,Pattern Building,Pattern Recognition,Modular Board,Network and Route Building,Point to Point Movement,Melding and Splaying,Negotiation,Trading,Push Your Luck,Income,Race,Random Production,Variable Set-up,Roll / Spin and Move,Variable Player Powers,Action Queue,Bias,Grid Movement,Lose a Turn,Programmed Movement,Scenario / Mission / Campaign Game,Voting,Events,Paper-and-Pencil,Player Elimination,Role Playing,Movement Points,Simulation,Variable Phase Order,Area Movement,Commodity Speculation,Cooperative Game,Deduction,Sudden Death Ending,Connections,Highest-Lowest Scoring,Betting and Bluffing,Memory,Score-and-Reset Game,Layering,Map Addition,Secret Unit Deployment,Increase Value of Unchosen Resources,Ratio / Combat Results Table,Take That,Team-Based Game,Campaign / Battle Card Driven,Tech Trees / Tech Tracks,Player Judge,Chit-Pull System,Three Dimensional Movement,Action Drafting,Minimap Resolution,Stat Check Resolution,Action Timer,Pick-up and Deliver,Map Deformation,Bingo,Crayon Rail System,Multiple Maps,Hidden Roles,Line Drawing,Tug of War,Pattern Movement,Static Capture,Different Dice Movement,Chaining,Ladder Climbing,Predictive Bid,Solo / Solitaire Game,Line of Sight,Critical Hits and Failures,Interrupts,Zone of Control,Bribery,End Game Bonuses,Area-Impulse,Worker Placement,Measurement Movement,Map Reduction,Real-Time,Resource to Move,Mancala,Ownership,Kill Steal,Hidden Movement,Track Movement,Deck Construction,Drafting,TableauBuilding,Prisoner's Dilemma,Hidden Victory Points,Movement Template,Slide/Push,Targeted Clues,Command Cards,Grid Coverage,Relative Movement,Action/Event,Card Play Conflict Resolution,"I Cut, You Choose",Die Icon Resolution,Elapsed Real Time Ending,Advantage Token,Storytelling,Catch the Leader,Roles with Asymmetric Information,Traitor Game,Moving Multiple Units,Semi-Cooperative Game,Communication Limits,Time Track,Speed Matching,Cube Tower,Re-rolling and Locking,Impulse Movement,Loans,Delayed Purchase,"Deck, Bag, and Pool Building",Move Through Deck,Single Loser Game,Matching,Induction,Physical Removal,Narrative Choice / Paragraph,Pieces as Map,Follow,Finale Ending,Order Counters,Contracts,Passed Action Token,King of the Hill,Action Retrieval,Force Commitment,Rondel,Automatic Resource Growth,Legacy Game,Dexterity,Physical
0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Load and Clean Subcategories

In [36]:
themes_expanded = pd.get_dummies(themes)
theme_sort = pd.DataFrame(themes_expanded.sum().sort_values(ascending=False))
themes_over_1 = list(theme_sort.loc[theme_sort[0]>1].index)
themes_attach = themes_expanded[themes_over_1]
themes_attach

,Theme_Food / Cooking,Theme_Superheroes,Theme_Anime / Manga,Theme_Cthulhu Mythos,Theme_Alternate History,Theme_Anthropomorphic Animals,Theme_Vikings,Theme_Post-Apocalyptic,Theme_Time Travel,Theme_Colonial,Theme_Mystery / Cri,Theme_Robots,Theme_Retro,Theme_Mad Science / Mad Scientist,Theme_Mining,Theme_Art,Theme_Archaeology / Paleontology,Theme_Witches,Theme_Deserts,Theme_Tropical,Theme_Steampunk,Theme_Gardening,Theme_Sieg,Theme_Flowers,Theme_Natur,Theme_Native Americans / First Peoples,Theme_Circus,Theme_Ninjas,Theme_King Arthur / The Knights of the Round Table / Camelot,Theme_Cyberpunk,Theme_Submarines,Theme_Construction,Theme_Samurai,Theme_Fantasy Sports,Theme_Love / Romanc,Theme_Biology,Theme_Kaiju,Theme_Gladiators,Theme_City,Theme_Villainy,Theme_Weather,Theme_Amusement Parks / Theme Parks,Theme_Airships / Blimps / Dirigibles / Zeppelins,Theme_Spooky Old Houses,Theme_Mech Warfar,Theme_Polic,Theme_Books / Libraries,Theme_Alchemy,Theme_Tropical Islands,Theme_Boardgaming,Theme_Oil / Gas / Petroleu,Theme_Arcade Video Games,Theme_Safaris,Theme_Dreams / Nightmares,Theme_Evolution,Theme_Survival,Theme_Fruit,Theme_Jail / Prison (Modern),Theme_Movie Industry,Theme_Animal Battles,Theme_Firefighting,Theme_Video Game Theme: Nintendo,Theme_Climate Chang,Theme_Helicopters,Theme_Psychology,Theme_Sci-Fi Sports,Theme_Sewing / Knitting / Cloth-Making,Theme_Beaches,Theme_Under the Sea,Theme_Aztecs,Theme_Trees and Forests,Theme_Mayans,Theme_Chibis,Theme_US National Parks,Theme_Scienc,Theme_Chemistry,Theme_Volcanoes,Theme_Fictional Games,Theme_Druids,Theme_Music Making & Makers,Theme_Psychic Powers,Theme_Trash / Garbag,Theme_Silk Road,Theme_Cereal Games,Theme_Automotive Industry,Theme_Video Game Theme: Tetris,Theme_TV Detectives,Theme_Romance of the Three Kingdoms,Theme_Trucks,Theme_Hanseatic Leagu,Theme_Chivalry / Jousting / Tournaments (Medieval Europe),Theme_Cannibals / Cannibalis,Theme_Canals,Theme_My Best Lif,Theme_Nuclear option,Theme_Astronomy,Theme_Cemeteries / Graveyards,Theme_Movies,Theme_Hackers,"Theme_Pub, Bars, Bistros...",Theme_Jewelry,Theme_School / College / University,Theme_UFOs,Theme_Templ,Theme_Mail / Stamps / The Post Offic,Theme_Memes,Theme_Computer / Information Technology Industry,Theme_Attorneys / Courts,Theme_Hot Air Balloons,Theme_Journalis,Theme_Motorcycles,Theme_Rubik's Cub,Theme_Dolls,Theme_Traffic / Driving,Theme_FIFA World Cup,Theme_Video Game Theme: Pokémon,Theme_Floating islands in the sky,Theme_Dieselpunk,Theme_Disney Theme Parks,Theme_Video Game Theme: Resident Evil,Theme_Video Game Theme: SEGA,Theme_Teaching Programming,Theme_Battle Royal,Theme_Earthquakes,Theme_Bacteria,Theme_Painting / Paintings,Theme_Television (TV) Industry,Theme_Knights Templar,Theme_African Americans,Theme_Hell,Theme_Tiki Cultur,Theme_Astrology,Theme_Video Game Theme: The Oregon Trail,Theme_Māori,Theme_Video Game Theme: Super Mario Bros.,Theme_Mushrooms,Theme_Pulp,Theme_Video Game Theme: Minecraft,Theme_Video Game Theme: Final Fantasy,Theme_Video Game Theme: Carmen Sandiego,Theme_Care Bears,Theme_Hike-Hiking,Theme_Inuit Peoples,Theme_Perfu,Theme_Camping,Theme_Latin American Political Games,Theme_Spanish Political Games,Theme_Video Game Theme: Doo,Theme_Fashion,Theme_Geocaching,Theme_Ecology,Theme_Chernobyl,Theme_Photography,Theme_French Foreign Legion,Theme_Cruise ships,Theme_Apache Tribes,Theme_Rivers,Theme_Flags identification
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
columns = themes_attach.columns

# call integer_reduce on the sets
themes_attach = integer_reduce(themes_attach, columns, fill_value=0)

Theme_Food / Cooking
Theme_Superheroes
Theme_Anime / Manga
Theme_Cthulhu Mythos
Theme_Alternate History
Theme_Anthropomorphic Animals
Theme_Vikings
Theme_Post-Apocalyptic
Theme_Time Travel
Theme_Colonial
Theme_Mystery / Cri
Theme_Robots
Theme_Retro
Theme_Mad Science / Mad Scientist
Theme_Mining
Theme_Art
Theme_Archaeology / Paleontology
Theme_Witches
Theme_Deserts
Theme_Tropical
Theme_Steampunk
Theme_Gardening
Theme_Sieg
Theme_Flowers
Theme_Natur
Theme_Native Americans / First Peoples
Theme_Circus
Theme_Ninjas
Theme_King Arthur / The Knights of the Round Table / Camelot
Theme_Cyberpunk
Theme_Submarines
Theme_Construction
Theme_Samurai
Theme_Fantasy Sports
Theme_Love / Romanc
Theme_Biology
Theme_Kaiju
Theme_Gladiators
Theme_City
Theme_Villainy
Theme_Weather
Theme_Amusement Parks / Theme Parks
Theme_Airships / Blimps / Dirigibles / Zeppelins
Theme_Spooky Old Houses
Theme_Mech Warfar
Theme_Polic
Theme_Books / Libraries
Theme_Alchemy
Theme_Tropical Islands
Theme_Boardgaming
Theme_Oil / Gas

In [38]:
# load subcategories file and check memory usage

indices = list(games['BGGId'])
subcategories = pd.read_pickle('data_dirty/subcategories.pkl')
subcategories.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21925 entries, 0 to 1
Data columns (total 85 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   BGGId                       21925 non-null  object 
 1   Economic                    1519 non-null   float64
 2   Negotiation                 665 non-null    float64
 3   Political                   497 non-null    float64
 4   Card Game                   6480 non-null   float64
 5   Fantasy                     2702 non-null   float64
 6   Abstract Strategy           1573 non-null   float64
 7   Medieval                    1052 non-null   float64
 8   Ancient                     737 non-null    float64
 9   Territory Building          514 non-null    float64
 10  Civilization                350 non-null    float64
 11  Nautical                    645 non-null    float64
 12  Exploration                 897 non-null    float64
 13  Travel                      241 non

In [39]:
subcategories.reset_index(inplace=True, drop=True) # reset index

In [40]:
subcategories.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
subcategories.fillna(0, inplace=True) # fill nan
#subcategories.set_index('BGGId', drop=True, inplace=True) # drop bad games
#subcategories = subcategories.loc[indices] # use only indices in games file

columns = subcategories.columns

# call integer_reduce on the sets
subcategories = integer_reduce(subcategories, columns, fill_value=0)


subcategories

BGGId
Economic
Negotiation
Political
Card Game
Fantasy
Abstract Strategy
Medieval
Ancient
Territory Building
Civilization
Nautical
Exploration
Travel
Farming
Mythology
Bluffing
Science Fiction
Collectible Components
Dice
Fighting
Print & Play
Maze
Miniatures
Racing
American West
City Building
Wargame
Adventure
Space Exploration
Renaissance
Modern Warfare
Humor
Electronic
Horror
Novel-based
Deduction
Word Game
Aviation / Flight
Movies / TV / Radio theme
Party Game
Memory
Puzzle
Real-time
Trivia
Industry / Manufacturing
World War II
American Civil War
Age of Reason
World War I
Trains
Animals
Children's Game
Pirates
Murder/Mystery
Transportation
Prehistoric
Sports
Action / Dexterity
Game System
Spies/Secret Agents
Educational
Medical
Mafia
Zombies
Comic Book / Strip
Napoleonic
Civil War
American Indian Wars
American Revolutionary War
Post-Napoleonic
Math
Book
Music
Environmental
Arabian
Number
Religious
Pike and Shot
Video Game Theme
Mature / Adult
Vietnam War
Korean War
Expansion for Bas

,BGGId,Economic,Negotiation,Political,Card Game,Fantasy,Abstract Strategy,Medieval,Ancient,Territory Building,Civilization,Nautical,Exploration,Travel,Farming,Mythology,Bluffing,Science Fiction,Collectible Components,Dice,Fighting,Print & Play,Maze,Miniatures,Racing,American West,City Building,Wargame,Adventure,Space Exploration,Renaissance,Modern Warfare,Humor,Electronic,Horror,Novel-based,Deduction,Word Game,Aviation / Flight,Movies / TV / Radio theme,Party Game,Memory,Puzzle,Real-time,Trivia,Industry / Manufacturing,World War II,American Civil War,Age of Reason,World War I,Trains,Animals,Children's Game,Pirates,Murder/Mystery,Transportation,Prehistoric,Sports,Action / Dexterity,Game System,Spies/Secret Agents,Educational,Medical,Mafia,Zombies,Comic Book / Strip,Napoleonic,Civil War,American Indian Wars,American Revolutionary War,Post-Napoleonic,Math,Book,Music,Environmental,Arabian,Number,Religious,Pike and Shot,Video Game Theme,Mature / Adult,Vietnam War,Korean War,Expansion for Base-game,Fan Expansion
0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21921,347521,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21922,348955,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21923,349131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Clean/Combine Mechanics and Subcategories

Manually cleaning up Subcategories. This section on BGG has a lot of "catch-all" concept that involve theming, mechanics (which should be in the mechanics section), and large subcategories that should be alone. We use our domain knowledge to clean this section.

In [41]:
# print our subcategories so we can prepare to sort them
subcategories.columns

Index(['BGGId', 'Economic', 'Negotiation', 'Political', 'Card Game', 'Fantasy',
       'Abstract Strategy', 'Medieval', 'Ancient', 'Territory Building',
       'Civilization', 'Nautical', 'Exploration', 'Travel', 'Farming',
       'Mythology', 'Bluffing', 'Science Fiction', 'Collectible Components',
       'Dice', 'Fighting', 'Print & Play', 'Maze', 'Miniatures', 'Racing',
       'American West', 'City Building', 'Wargame', 'Adventure',
       'Space Exploration', 'Renaissance', 'Modern Warfare', 'Humor',
       'Electronic', 'Horror', 'Novel-based', 'Deduction', 'Word Game',
       'Aviation / Flight', 'Movies / TV / Radio theme', 'Party Game',
       'Memory', 'Puzzle', 'Real-time', 'Trivia', 'Industry / Manufacturing',
       'World War II', 'American Civil War', 'Age of Reason', 'World War I',
       'Trains', 'Animals', 'Children's Game', 'Pirates', 'Murder/Mystery',
       'Transportation', 'Prehistoric', 'Sports', 'Action / Dexterity',
       'Game System', 'Spies/Secret Agents'

In [42]:
# picking the items that will go under "themes"
subcats_themes = ['BGGId', 'Adventure', 'Fantasy', 'Fighting', 'Environmental', 'Medical', 'Economic',
       'Industry / Manufacturing', 'Transportation', 'Science Fiction',
       'Space Exploration', 'Civilization', 'Civil War',
       'Movies / TV / Radio theme', 'Novel-based',
       'Age of Reason', 'Mythology', 'Renaissance', 'American West', 'Animals',
       'Modern Warfare', 'Medieval', 'Ancient','Nautical', 'Post-Napoleonic', 'Horror',
         'Farming', 'Religious', 'Travel',
       'Murder/Mystery', 'Pirates', 'Comic Book / Strip', 'Mature / Adult',
       'Video Game Theme', 'Spies/Secret Agents','Arabian', 'Prehistoric',
                 'Trains','Aviation / Flight', 'Zombies',
       'World War II', 'Racing', 'Pike and Shot', 'World War I','Humor','Sports',
                 'Mafia','American Indian Wars', 'Napoleonic',
       'American Revolutionary War','Vietnam War', 'American Civil War','Number', 'Trivia',
       'Music', 'Korean War', 'City Building', 'Political', 'Math', 'Maze',]

In [43]:
# picking the items that will stay as subcategories
subcats_subcategories = ['BGGId', 'Exploration', 'Miniatures', 'Territory Building', 'Card Game',
             'Educational', 'Puzzle','Collectible Components',
        'Word Game','Print & Play','Electronic']

In [44]:
# looking at our mechanics in alphabetical order so we can see where to roll in the subcategory mechanics
sorted(mechanics.columns)

['Action Drafting',
 'Action Points',
 'Action Queue',
 'Action Retrieval',
 'Action Timer',
 'Action/Event',
 'Advantage Token',
 'Alliances',
 'Area Majority / Influence',
 'Area Movement',
 'Area-Impulse',
 'Auction/Bidding',
 'Automatic Resource Growth',
 'BGGId',
 'Betting and Bluffing',
 'Bias',
 'Bingo',
 'Bribery',
 'Campaign / Battle Card Driven',
 'Card Play Conflict Resolution',
 'Catch the Leader',
 'Chaining',
 'Chit-Pull System',
 'Command Cards',
 'Commodity Speculation',
 'Communication Limits',
 'Connections',
 'Contracts',
 'Cooperative Game',
 'Crayon Rail System',
 'Critical Hits and Failures',
 'Cube Tower',
 'Deck Construction',
 'Deck, Bag, and Pool Building',
 'Deduction',
 'Delayed Purchase',
 'Dexterity',
 'Dice Rolling',
 'Die Icon Resolution',
 'Different Dice Movement',
 'Drafting',
 'Elapsed Real Time Ending',
 'Enclosure',
 'End Game Bonuses',
 'Events',
 'Finale Ending',
 'Follow',
 'Force Commitment',
 'Grid Coverage',
 'Grid Movement',
 'Hand Managemen

In [45]:
# drop subcategories that are none of the things
subcategories.drop(['Expansion for Base-game',  'Game System', 'Book'], axis=1, inplace=True)

In [46]:
# for mechanics that are mistakenly in the subcategories frame, make sure those mechanics are in the mechanics frame instead
# remove them from the subcategories frame after moving them over

indices = list(subcategories.loc[subcategories['Negotiation']==1].index)
mechanics.loc[indices, 'Negotiation']=1
subcategories.drop('Negotiation', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Action / Dexterity']==1].index)
mechanics.loc[indices, 'Dexterity']=1
subcategories.drop('Action / Dexterity', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Dice']==1].index)
mechanics.loc[indices, 'Dice Rolling']=1
subcategories.drop('Dice', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Bluffing']==1].index)
mechanics.loc[indices, 'Betting and Bluffing']=1
subcategories.drop('Bluffing', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Real-time']==1].index)
mechanics.loc[indices, 'Real-Time']=1
subcategories.drop('Real-time', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Memory']==1].index)
mechanics.loc[indices, 'Memory']=1
subcategories.drop('Memory', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Deduction']==1].index)
mechanics.loc[indices, 'Deduction']=1
subcategories.drop('Deduction', axis=1, inplace=True)


In [47]:
# or for larger categories in the subcategories frame, move those to the games frame
# remove them from the subcategories frame after moving them over

indices = list(subcategories.loc[subcategories['Wargame']==1].index)
games.loc[indices, 'Cat:War']=1
subcategories.drop('Wargame', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories["Children's Game"]==1].index)
games.loc[indices, 'Cat:Childrens']=1
subcategories.drop("Children's Game", axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Party Game']==1].index)
games.loc[indices, 'Cat:Party']=1
subcategories.drop('Party Game', axis=1, inplace=True)

indices = list(subcategories.loc[subcategories['Abstract Strategy']==1].index)
games.loc[indices, 'Cat:Abstract']=1
subcategories.drop('Abstract Strategy', axis=1, inplace=True)

In [48]:
# make new data frames for THEMES and SUBCATEGORIES
themes = subcategories[subcats_themes]
themes = pd.concat([themes, themes_attach], axis=1)
subcategories = subcategories[subcats_subcategories]

Save all the file we just cleaned or created!

In [ ]:
mechanics.to_pickle('data_cleaned/mechanics.pkl')

In [ ]:
subcategories.to_pickle('data_cleaned/subcategories.pkl')

In [ ]:
themes.to_pickle('data_cleaned/themes.pkl')

In [49]:
mechanics.to_csv('data_kaggle/mechanics.csv', index=False)
subcategories.to_csv('data_kaggle/subcategories.csv', index=False)
themes.to_csv('data_kaggle/themes.csv', index=False)

In [29]:
mechanics = pd.read_pickle('data_cleaned/mechanics.pkl')
subcategories = pd.read_pickle('data_cleaned/subcategories.pkl')
themes = pd.read_pickle('data_cleaned/themes.pkl')

In [30]:
mechanics.head()

,BGGId,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,Set Collection,Tile Placement,Action Points,Investment,Market,Square Grid,Stock Holding,Victory Points as a Resource,Enclosure,Pattern Building,Pattern Recognition,Modular Board,Network and Route Building,Point to Point Movement,Melding and Splaying,Negotiation,Trading,Push Your Luck,Income,Race,Random Production,Variable Set-up,Roll / Spin and Move,Variable Player Powers,Action Queue,Bias,Grid Movement,Lose a Turn,Programmed Movement,Scenario / Mission / Campaign Game,Voting,Events,Paper-and-Pencil,Player Elimination,Role Playing,Movement Points,Simulation,Variable Phase Order,Area Movement,Commodity Speculation,Cooperative Game,Deduction,Sudden Death Ending,Connections,Highest-Lowest Scoring,Betting and Bluffing,Memory,Score-and-Reset Game,Layering,Map Addition,Secret Unit Deployment,Increase Value of Unchosen Resources,Ratio / Combat Results Table,Take That,Team-Based Game,Campaign / Battle Card Driven,Tech Trees / Tech Tracks,Player Judge,Chit-Pull System,Three Dimensional Movement,Action Drafting,Minimap Resolution,Stat Check Resolution,Action Timer,Pick-up and Deliver,Map Deformation,Bingo,Crayon Rail System,Multiple Maps,Hidden Roles,Line Drawing,Tug of War,Pattern Movement,Static Capture,Different Dice Movement,Chaining,Ladder Climbing,Predictive Bid,Solo / Solitaire Game,Line of Sight,Critical Hits and Failures,Interrupts,Zone of Control,Bribery,End Game Bonuses,Area-Impulse,Worker Placement,Measurement Movement,Map Reduction,Real-Time,Resource to Move,Mancala,Ownership,Kill Steal,Hidden Movement,Track Movement,Deck Construction,Drafting,TableauBuilding,Prisoner's Dilemma,Hidden Victory Points,Movement Template,Slide/Push,Targeted Clues,Command Cards,Grid Coverage,Relative Movement,Action/Event,Card Play Conflict Resolution,"I Cut, You Choose",Die Icon Resolution,Elapsed Real Time Ending,Advantage Token,Storytelling,Catch the Leader,Roles with Asymmetric Information,Traitor Game,Moving Multiple Units,Semi-Cooperative Game,Communication Limits,Time Track,Speed Matching,Cube Tower,Re-rolling and Locking,Impulse Movement,Loans,Delayed Purchase,"Deck, Bag, and Pool Building",Move Through Deck,Single Loser Game,Matching,Induction,Physical Removal,Narrative Choice / Paragraph,Pieces as Map,Follow,Finale Ending,Order Counters,Contracts,Passed Action Token,King of the Hill,Action Retrieval,Force Commitment,Rondel,Automatic Resource Growth,Legacy Game,Dexterity,Physical
0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
subcategories.head()

,BGGId,Exploration,Miniatures,Territory Building,Card Game,Educational,Puzzle,Collectible Components,Word Game,Print & Play,Electronic
0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,1,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0


In [32]:
themes.head()

,BGGId,Adventure,Fantasy,Fighting,Environmental,Medical,Economic,Industry / Manufacturing,Transportation,Science Fiction,Space Exploration,Civilization,Civil War,Movies / TV / Radio theme,Novel-based,Age of Reason,Mythology,Renaissance,American West,Animals,Modern Warfare,Medieval,Ancient,Nautical,Post-Napoleonic,Horror,Farming,Religious,Travel,Murder/Mystery,Pirates,Comic Book / Strip,Mature / Adult,Video Game Theme,Spies/Secret Agents,Arabian,Prehistoric,Trains,Aviation / Flight,Zombies,World War II,Racing,Pike and Shot,World War I,Humor,Sports,Mafia,American Indian Wars,Napoleonic,American Revolutionary War,Vietnam War,American Civil War,Number,Trivia,Music,Korean War,City Building,Political,Math,Maze,Theme_Food / Cooking,Theme_Superheroes,Theme_Anime / Manga,Theme_Cthulhu Mythos,Theme_Alternate History,Theme_Anthropomorphic Animals,Theme_Vikings,Theme_Post-Apocalyptic,Theme_Time Travel,Theme_Colonial,Theme_Mystery / Cri,Theme_Robots,Theme_Retro,Theme_Mad Science / Mad Scientist,Theme_Mining,Theme_Art,Theme_Archaeology / Paleontology,Theme_Witches,Theme_Deserts,Theme_Tropical,Theme_Steampunk,Theme_Gardening,Theme_Sieg,Theme_Flowers,Theme_Natur,Theme_Native Americans / First Peoples,Theme_Circus,Theme_Ninjas,Theme_King Arthur / The Knights of the Round Table / Camelot,Theme_Cyberpunk,Theme_Submarines,Theme_Construction,Theme_Samurai,Theme_Fantasy Sports,Theme_Love / Romanc,Theme_Biology,Theme_Kaiju,Theme_Gladiators,Theme_City,Theme_Villainy,Theme_Weather,Theme_Amusement Parks / Theme Parks,Theme_Airships / Blimps / Dirigibles / Zeppelins,Theme_Spooky Old Houses,Theme_Mech Warfar,Theme_Polic,Theme_Books / Libraries,Theme_Alchemy,Theme_Tropical Islands,Theme_Boardgaming,Theme_Oil / Gas / Petroleu,Theme_Arcade Video Games,Theme_Safaris,Theme_Dreams / Nightmares,Theme_Evolution,Theme_Survival,Theme_Fruit,Theme_Jail / Prison (Modern),Theme_Movie Industry,Theme_Animal Battles,Theme_Firefighting,Theme_Video Game Theme: Nintendo,Theme_Climate Chang,Theme_Helicopters,Theme_Psychology,Theme_Sci-Fi Sports,Theme_Sewing / Knitting / Cloth-Making,Theme_Beaches,Theme_Under the Sea,Theme_Aztecs,Theme_Trees and Forests,Theme_Mayans,Theme_Chibis,Theme_US National Parks,Theme_Scienc,Theme_Chemistry,Theme_Volcanoes,Theme_Fictional Games,Theme_Druids,Theme_Music Making & Makers,Theme_Psychic Powers,Theme_Trash / Garbag,Theme_Silk Road,Theme_Cereal Games,Theme_Automotive Industry,Theme_Video Game Theme: Tetris,Theme_TV Detectives,Theme_Romance of the Three Kingdoms,Theme_Trucks,Theme_Hanseatic Leagu,Theme_Chivalry / Jousting / Tournaments (Medieval Europe),Theme_Cannibals / Cannibalis,Theme_Canals,Theme_My Best Lif,Theme_Nuclear option,Theme_Astronomy,Theme_Cemeteries / Graveyards,Theme_Movies,Theme_Hackers,"Theme_Pub, Bars, Bistros...",Theme_Jewelry,Theme_School / College / University,Theme_UFOs,Theme_Templ,Theme_Mail / Stamps / The Post Offic,Theme_Memes,Theme_Computer / Information Technology Industry,Theme_Attorneys / Courts,Theme_Hot Air Balloons,Theme_Journalis,Theme_Motorcycles,Theme_Rubik's Cub,Theme_Dolls,Theme_Traffic / Driving,Theme_FIFA World Cup,Theme_Video Game Theme: Pokémon,Theme_Floating islands in the sky,Theme_Dieselpunk,Theme_Disney Theme Parks,Theme_Video Game Theme: Resident Evil,Theme_Video Game Theme: SEGA,Theme_Teaching Programming,Theme_Battle Royal,Theme_Earthquakes,Theme_Bacteria,Theme_Painting / Paintings,Theme_Television (TV) Industry,Theme_Knights Templar,Theme_African Americans,Theme_Hell,Theme_Tiki Cultur,Theme_Astrology,Theme_Video Game Theme: The Oregon Trail,Theme_Māori,Theme_Video Game Theme: Super Mario Bros.,Theme_Mushrooms,Theme_Pulp,Theme_Video Game Theme: Minecraft,Theme_Video Game Theme: Final Fantasy,Theme_Video Game Theme: Carmen Sandiego,Theme_Care Bears,Theme_Hike-Hiking,Theme_Inuit Peoples,Theme_Perfu,Theme_Camping,Theme_Latin American Political Games,Theme_Spanish Political Games,Theme_Video Game Theme: Doo,Theme_Fashion,Theme_Geocaching,Theme_Ecology,Theme_Chernobyl,Theme_Photography,Theme_French 

## Designers

In [50]:
# Load up our designers file!
designers = pd.read_pickle('data_dirty/designers.pkl')
designers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21925 entries, 0 to 1
Columns: 9808 entries, BGGId to Alexandre Aguilar
dtypes: float64(9807), object(1)
memory usage: 1.6+ GB


In [51]:
designers.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
designers.fillna(0, inplace=True) # Fill NaN

# change all flags to int8
temp_id = designers['BGGId'] 
designers.drop('BGGId', axis=1, inplace=True)
designers = designers.astype('int8')
designers['BGGId'] = temp_id

designers.reset_index(inplace=True, drop=True)# reset index
designers.info() # check memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Columns: 9808 entries, Karl-Heinz Schmiel to BGGId
dtypes: int64(1), int8(9807)
memory usage: 205.2 MB


In [52]:
# save file of all possible designers
designers.to_pickle('data_cleaned/designers_all.pkl')
designers.to_csv('data_kaggle/designers_all.csv', index=False)

In [53]:
# locate all row that sum to 3 or less (find low experience designers)

# change the 3 to whatever desired for more or less experience
lowexp_rows = designers.loc[:, designers.sum(axis=0) <= 3]

# Locate the columns that contain the low experience designers
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp designer
designers['Low-Exp Designer'] = 0

# for each index in the low exp list, set low exp designer to 1
for index in indices:
    designers.loc[index, 'Low-Exp Designer'] = 1
    
# drop all columns for one-off designers
designers.drop(designers.loc[:, designers.sum(axis=0) <= 3], axis=1, inplace=True)

In [54]:
# save file of designers reduced to 3 or more works
designers.to_pickle('data_cleaned/designers_reduced.pkl')
designers.to_csv('data_kaggle/designers_reduced.csv', index=False)

In [33]:
designers = pd.read_pickle('data_cleaned/designers_reduced.pkl')
designers.head()

,Karl-Heinz Schmiel,"G. W. ""Jerry"" D'Arcey",Reiner Knizia,Sid Sackson,Jean du Poël,Martin Wallace,Richard Ulrich,Wolfgang Kramer,Alan R. Moon,Uwe Rosenberg,Klaus Teuber,Reinhard Staupe,Bill Eberle,Bill Norton,Jack Kittredge,Peter Olotka,Stefan Dorra,James Ernest,Richard Garfield,Richard Hamblen,Glenn Rahman,Kenneth Rahman,Christian T. Petersen,Don Greenwood,Steve Jackson (I),Eamon Bloomfield,Alex Randolph,Lynn Willis,Richard Launius,Sandy Petersen,Stephen V. Cole,Ian Livingstone,Richard Borg,Nigel Stillman,Rick Priestley,Valentin Herman,Michael Kiesling,Yves Hirschfeld,Doug Kaufman,Tom Wham,Philippe Keyaerts,Klaus Palesch,Stephen Hand,Dan Campagna,Pascal Bernard,Mark Simonitch,Richard H. Berg,Franz-Benno Delonge,Francis Tresham,Marcel-André Casasola Merkle,Dirk Henn,Mark Alan Osterhaus,Matthew Kirby,Craig Besinque,Tom Dalgliesh,Gary Selkirk,Lance Gutteridge,Grant Dalgliesh,Craig Van Ness,Ted Raicer,Doris Matthäus,Frank Nestel,"Larry Harris, Jr.",Michael Gray,Mike Fitzgerald,Kris Burm,Dominique Ehrhard,Duccio Vitale,Justin Kemppainen,Paul Peterson,Niek Neuwahl,Mark McLaughlin,Peter Burley,Darwin Bromley,Friedemann Friese,Thorsten Gimmler,Michael Schacht,Juan Rodriguez,Roger Heyworth,Sylvie Barc,"Edward Horn, Jr.",Christwart Conrad,Klaus Kreowski,Reinhold Wittig,Dieter Danziger,Larry Roznai,Eric Solomon,Bill Fawcett,(Uncredited),Eric Randall,Laurent Lavaur,Hanno Kuhn,Wilfried Kuhn,Albert Lamorisse,Michael I. Levin,David Hoyt,Reiner Stockhausen,Horst-Rainer Rösner,Urs Hostettler,James Kyle,Andreas Seyfarth,Jim Lin,Skaff Elias,Rüdiger Dorn,Rudi Hoffmann,Gary Dicken,Phil Kendall,Steve Kendall,Andrew Looney,John Cooper,Corné van Moorsel,S. Craig Taylor,Irad B. Hardy,Lewis Pulsipher,Thomas Lehmann,Eric Goldberg,Michael S. Matheny,Michele Quondam,Derek Carver,Peter Hawes,John Young,Greg Pinder,Harry Rowland,Tom Walczyk,Kristin Looney,Ben Knight,Günter Burkhardt,Dan Glimne,Bernhard Weber,Francis Pacherie,Kevin Zucker,Nicolas Pilartz,Bruce Harper,Phil Eklund,François-Stanislas Thomas,Charles Vasey,Philippe Mouchebeuf,Jacques Zeimet,Richard Breese,Leo Colovini,Philip C. Barker,Richard Bodley Scott,Hartmut Witt,Günter Cornett,Garrett J. Donner,Michael S. Steer,Mick Uhl,Bertram Kaes,Virginia Charves,Gerhard E. Kodys,Spartaco Albertarelli,John Bohrer,Neil Zimmerer,Richard Halliwell,Charles Phillips,Marsha J. Falco,Phil Orbanes Sr.,Hermann Huber,Scott Kimball,David Parlett,Henri Sala,Frank DiLorenzo,Michael Rieneck,Bruno Faidutti,Thomas Fackler,Darrell Hardy,Tom Jolly,Bernhard Naegele,Joris Wiersinga,Bill Dixon,Bruno Sinigaglio,H. Jean Vanaise,Frederick A. Herschler,Max J. Kobbert,Robert Abbott,Nik Sewell,Mark Sienholz,Ann Stambler,Monty Stambler,Wolfgang Riedesser,David G. Watts,Ralf zur Linde,Philippe des Pallières,Jakob Bonds,Wolfgang Panning,John Prados,Hartmut Kommerell,Heinz Meister,Angelo Porazzi,Aaron Weissblum,Johann Rüttinger,John Harshman,Wolfgang A. Lehmann,Michel Lalet,Walter Müller,Angelo Zucca,Jeff Grubb,Jeroen Doumen,Jon Darbro,Klaus Zoch,Thilo Hutzler,Terry Goodchild,Courtney F. Allen,Tom Kremer,Guido Hoffmann,Torsten Marold,Mark Herman,Brian Snoddy,Roberto Fraga,Brian Hersch,Eric M. Lang,Serge Laget,Tom Schoeps,Randall C. Reed,Randall Horn,Lester Smith,Brad Stock,Joli Quentin Kansil,Roland Siegers,Charles T. Kamps,David James Ritchie,R. Hunter Gough,Stephen Baker,Alan Emrich,Frank Chadwick,Mark Acres,Joseph Miranda,Edward Curran,Frank Davis,Douglas Niles,Jervis Johnson,Douglas Malewicki,Robert Harris,"David ""Zeb"" Cook",Andreas Wetter,Thorsten Löpmann,John Astell,Jim Dunnigan,Randy Heller,Matt Leacock,Tim Walsh,Frank Thibault,Hajo Bücken,Alfred Leonardi,Jim Keifer,Rob Daviau,Klaus-Jürgen Wrede,Dan Smith,Peter Wichmann,Dean Essig,William Sariego,Jordan Weisman,Kevin Barrett,Martin Schlegel,Peter Eggert,Richard Tait,Whit Alexander,Anthony F. Buccini,Redmond A. Simonsen,Dan Verssen,Frank Branham,Thomas N. Shaw,Max Michael,Andrew McNeil,Bruce Shelley,John Hill (I),Dave Williams (II),Edith Grein-Böttcher,Zoltán Aczél,Roger Fo

## Artists

In [55]:
# load artists file
artists = pd.read_pickle('data_dirty/artists.pkl')
artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21925 entries, 0 to 1
Columns: 10987 entries, BGGId to Stephan Lorenz
dtypes: float64(10986), object(1)
memory usage: 1.8+ GB


In [56]:
artists.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
artists.fillna(0, inplace=True) # fill nan

# convert flags to int8
temp_id = artists['BGGId']
artists.drop('BGGId', axis=1, inplace=True)
artists = artists.astype('int8')
artists['BGGId'] = temp_id

artists.reset_index(inplace=True, drop=True)# reset index
artists.info()# get memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Columns: 10987 entries, Harald Lieske to BGGId
dtypes: int64(1), int8(10986)
memory usage: 229.9 MB


In [57]:
# save all artists to file
artists.to_pickle('data_cleaned/artists_all.pkl')
artists.to_csv('data_kaggle/artists_all.csv', index=False)

In [58]:
# locate all row that sum to 3 or less (find low experience artists)

# change the 3 to whatever desired for more or less experience
lowexp_rows = artists.loc[:, artists.sum(axis=0) <= 3]

# Locate the columns that contain the low experience artists
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp Artist
artists['Low-Exp Artist'] = 0

# for each index in the low exp list, set low exp Artist to 1
for index in indices:
    artists.loc[index, 'Low-Exp Artist'] = 1
    
# drop all columns for one-off artists
artists.drop(artists.loc[:, artists.sum(axis=0) <= 3], axis=1, inplace=True)

In [59]:
# save artists to file reduces to 3 or more works
artists.to_pickle('data_cleaned/artists_reduced.pkl')
artists.to_csv('data_kaggle/artists_reduced.csv', index=False)

In [34]:
artists = pd.read_pickle('data_cleaned/artists_reduced.pkl')
artists.head()

,Harald Lieske,Franz Vohwinkel,Peter Whitley,Scott Okumura,(Uncredited),Doris Matthäus,Alan R. Moon,Alexander Jung,Andrea Boekhoff,Björn Pertoft,Fréderic Bertrand,Guido Hoffmann,Johann Rüttinger,Klemens Franz,Marek Bláha,Markus Wagner,Oliver Freudenreich,Allison Kline,Ian O'Toole,Jason Hawkins,Marion Pott,Michael Menzel,Michaela Kienle,Pete Fenlon,Stephen Graham Walsh,Tanja Donner,Volkan Baga,John Kovalic,Lüders & Baran,Anders Jeppsson,John Blanche,Brian Snoddy,Cheyenne Wright,Gerald Brom,Larry Elmore,Michelle Nephew,Phil Foglio,Daniel Gelon,Peter Bergting,Pascal Trigaux,Chris White (I),Randall C. Reed,Richard Hamblen,David A. Trampier,"David C. Sutherland, III",Kenneth Rahman,Christian T. Petersen,Kurt Miller,Stephen Langmead,Dan Smith,Dave Martin,Frank Gerwin,Jeffrey D. George,Lar DeSouza,Chris Baker,Dahee Lee,David Martin (II),"William H. Keith, Jr.",Christos Achilleos,Franz-Josef Schulte,Linda Bound,Atelier Wilinski,Heiko Günther,Klaus Wilinski,Walter Pepperle,Bascu,Christine Conrad,Tom Thiel,Alexander Bradley,Bildgården AB,Design Edge,Thierry Masson,David Cherry,Dominique Ehrhard,Vincent Dutrait,Mathieu Leyssenne,Anke Pohl,Claus Stephan,Thilo Rick,Dave Andrews,John Sibbick,Kevin Walker,Martin McKenna,Paul Bonner,Russ Nicholson,Stephen Tappin,Tony Ackland,Christophe Swal,Mariusz Gandzel,Paul Mafayon,Charles Elliot (I),Kevin Wilkins,Martin Hoffmann,Richard A. Johnson,Tom Wham,Walter Velez,Cyril Saint Blancat,Aly Morrison,Gary Chalk,Arnold Hendrick,Greg Cervall,Andreas Steiner,Mark Simonitch,Charles Kibler,Dale Sheaffer,Ed Dovey,Guillaume Rohmer,Rodger B. MacGowan,Marcel-André Casasola Merkle,Jo Hartwig,Oliver Schlemmer,Reinhard Horst,James Ernest,Val Mayerik,Eric Hotz,Alessandra Cimatoribus,Terry Leeds,Kenn Nishiuye,Mike Doyle (I),Cyrille Daujean,Keith Parkinson,Virginijus Poshkus,lu'cifer,William O'Connor,John Goodenough,Samuel R. Shimota,Scott Nicely,Tomasz Jedruszek,別府さい (Sai Beppu),Mike Raabe,Quinton Hoover,Paul Laane,Anders Finér,Eric Lofgren,Henning Ludvigsen,Jim Pavelec,Kevin Childress,Peter Tikos,Ritva Hussain-Shahid,Jean Baer,Mick Uhl,Jessica Ney,Marek Szyszko,Paul Niemeyer,Steve Tolley,Thomas Weiss,Chris Vande Voort,Javier González Cava,"Lars-Arne ""Maura"" Kalusky",Olivier Frot,Simon Roberts,Olivier Fagnère,Alvin Madden,William Bricker,Reinhold Wittig,Ralf E. Kahlert,Gerhard Schmid,Mike Atkinson,Drew Perkett,Jim Clouse,Mike Russo,Peter Y. Bromley,Pierô,Antonio Dessi,David B. Bromley,Franck Dion,Néstor Romeral Andrés,Paolo Chiari,Bernard Bittler,Tobias Schweiger,Wilfried Gebhard,Elisa Teague,Paul Herbert,Wesly Gibs,Barbro Hennius,Nikola Kucharska,Thomas Vuarchex,Christof Tisch,Clay Gardner,Res Brandenberger,Angelo Porazzi,Jacqui Davis,TANSAN Inc.,Tony Rochon,Chris Quilliams,Christine Alcouffe,François Bruel,Kerri Aitken,Philippe Guérin,Brian Schomburg,Marko Djurdjević,Jochen Eeuwyk,Stefan Thulin,Mateusz Bielski,Yusuke Mamada,Steve Argyle,Marc Margielsky,"George I. Parrish, Jr.",Paweł Kurowski,Piotr Słaby,Dave Dobyski,Redmond A. Simonsen,Patrick McEvoy,Armand Cabrera,Carolyn Wales,Larry Catalano,Stephen Crane,Nicolás Eskubi,Colin Dixon,Matthias Catrein,Wolfgang Steinmeyer,Michael Fisher,James Ryman,Andrew Looney,Barbara Spelger,Michael Hays,Clément Masson,Eckhard Freytag,Ian Parovel,Renate Seelig,Charlie Bink,Matt Groening,Christopher Moeller,Howard David Johnson,Marko Fiedler,Peggy Coleman,Klaus Teuber,Beth Queman,David McElhannon,Pixel Productions Inc.,Oliver Richtberg,Juliet Breese,Katharina Kubisch,Matthias Wittig,Wolfgang Kramer,Colleen O'Malley,Dave LaForce,C. Leslie Crandall,Foto Rhomberg,Brett Ewins,Patrick LaMontagne,Jun Sasaki,Stéphane Poinsot,Per Sjöholm,Camille Kuo,Rolf Vogt,Pedro A. Alberto,Tilman Michalski,Jean-Baptiste Reynaud,C. Brent Ferguson,J. Scott Reeves,James Talbot,Ben Prenevost,Bradley McDevitt,Earl Geier,Louis Frank,Scott Schomburg,Tyler Walpole,Wolfgang Scheit,Gabriela Silveira,Jürgen Martens,Georg von Westphalen,Jared Blando,Mark Zug,Kinetic,Albert Uderzo,René Goscinny,J. C. Sohier,Andrea

## Publishers

In [60]:
# load publishers
publishers = pd.read_pickle('data_dirty/publishers.pkl')
publishers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21925 entries, 0 to 1
Columns: 7070 entries, BGGId to NISEI
dtypes: float64(7069), object(1)
memory usage: 1.2+ GB


In [61]:
publishers.drop_duplicates(subset='BGGId', keep='first', inplace=True) # drop duplicates
publishers.fillna(0, inplace=True)# fill nan

# set flags to int8
temp_id = publishers['BGGId']
publishers.drop('BGGId', axis=1, inplace=True)
publishers = publishers.astype('int8')
publishers['BGGId'] = temp_id

publishers.reset_index(inplace=True, drop=True) # reset index
publishers.info() # get memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Columns: 7070 entries, Hans im Glück to BGGId
dtypes: int64(1), int8(7069)
memory usage: 148.0 MB


In [62]:
# save all publishers to file
publishers.to_pickle('data_cleaned/publishers_all.pkl')
publishers.to_csv('data_kaggle/publishers_all.csv', index=False)

In [63]:
# locate all row that sum to 3 or less (find low experience publishers)

# change the 3 to whatever desired for more or less experience
lowexp_rows = publishers.loc[:, publishers.sum(axis=0) <= 3]

# Locate the columns that contain the low experience publishers
lowexp_columns = lowexp_rows[lowexp_rows.sum(axis=1) > 0]

# get indices of those low-exp columns
indices = lowexp_columns.index

# make new column for low exp Publisher
publishers['Low-Exp Publisher'] = 0

# for each index in the low exp list, set low exp Publisher to 1
for index in indices:
    publishers.loc[index, 'Low-Exp Publisher'] = 1
    
# drop all columns for one-off publishers
publishers.drop(publishers.loc[:, publishers.sum(axis=0) <= 3], axis=1, inplace=True)

In [64]:
# save publishers reduced to 3 or more works
publishers.to_pickle('data_cleaned/publishers_reduced.pkl')
publishers.to_csv('data_kaggle/publishers_reduced.csv', index=False)

In [35]:
publishers = pd.read_pickle('data_cleaned/publishers_reduced.pkl')
publishers.head()

Hans im Glück  Moskito Spiele  Portal Games  Spielworxx  Stronghold Games  \
0              1               1             1           1                 1   
1              0               0             0           0                 0   
2              1               0             0           0                 0   
3              0               0             0           0                 0   
4              0               0             0           0                 0   

   Valley Games, Inc.  YOKA Games  sternenschimmermeer  E.S. Lowe  \
0                   1           1                    1          0   
1                   0           0                    0          1   
2                   0           0                    0          0   
3                   0           0                    0          0   
4                   0           0                    0          0   

   Milton Bradley  999 Games  ABACUSSPIELE  Descartes Editeur  \
0               0          0             0                  0   
1               1          0             0                  0   
2               0          1             1                  1   
3               0          0             0                  0   
4               0          0             0                  0   

   Edge Entertainment  Fantasy Flight Games  Galakta  Hobby Japan  \
0                   0                     0        0            0   
1                   0                     0        0            0   
2                   1                     1        1            1   
3                   0                     0        0            0   
4                   0                     0        0            0   

   Korea Boardgames Co., Ltd.  Lacerta  Lautapelit.fi  Rio Grande Games  \
0                           0        0              0                 0   
1                           0        0              0                 0   
2                           1        1              1                 1   
3                           0        0              0                 0   
4                           0        0              0                 0   

   Smart Ltd  Wargames Club Publishing  KOSMOS  3M  Avalon Hill Games, Inc.  \
0          0                         0       0   0                        0   
1          0                         0       0   0                        0   
2          1                         1       0   0                        0   
3          0                         0       1   0                        0   
4          0                         0       0   1                        1   

   Dujardin  Grow Jogos e Brinquedos  PS-Games  Schmidt France  \
0         0                        0         0               0   
1         0                        0         0               0   
2         0                        0         0               0   
3         0                        0         0               0   
4         1                        1         1               1   

   Schmidt International  Schmidt Spiele  Selecta Spel en Hobby  \
0                      0               0                      0   
1                      0               0                      0   
2                      0               0                      0   
3                      0               0                      0   
4                      1               1                      1   

   The Avalon Hill Game Co  \
0                        0   
1                        0   
2                        0   
3                        0   
4                        1   

   Historien Spiele Galerie (Historien Spielegalerie)  Falomir Juegos  \
0                                                  0                0   
1                                                  0                0   
2                                                  0                0   
3                                                  0                0   
4                            

# Ratings - by Item, User, & Comments

## Test Code

In [65]:
master_comments = pd.read_pickle('data_dirty/raw_game_ratings.pkl')

# integer_reduce
master_comments['BGGId'] = master_comments['BGGId'].astype(int)
master_comments['Rating'] = master_comments['Rating'].astype(float)

master_comments.head()

,BGGId,Name,Username,Rating,Comments
0,213788,Ascension: Year Three Collector's Edition,Tonydorrf,8.0,
1,213788,Ascension: Year Three Collector's Edition,tachyon14k,8.0,
2,213788,Ascension: Year Three Collector's Edition,Ungotter,8.0,
3,213788,Ascension: Year Three Collector's Edition,brainlocki3,8.0,
4,213788,Ascension: Year Three Collector's Edition,PPMP,8.0,


In [66]:
master_comments.drop_duplicates(keep='first', inplace=True)

In [67]:
master_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18954963 entries, 0 to 62419
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int32  
 1   Name      object 
 2   Username  object 
 3   Rating    float64
 4   Comments  object 
dtypes: float64(1), int32(1), object(3)
memory usage: 795.4+ MB


In [68]:
nodrops = list(master_comments.loc[master_comments.Comments!=''].index)

In [69]:
comments_only = master_comments.iloc[nodrops]

In [70]:
comments_only.head(10)

,BGGId,Name,Username,Rating,Comments
5,213788,Ascension: Year Three Collector's Edition,Dychotimer,8.0,Excellent Ascension that now are all full sets...
14,213788,Ascension: Year Three Collector's Edition,Narstrand,7.8,Fantastic deck builder! The quality of the Col...
27,213788,Ascension: Year Three Collector's Edition,bassgonist,7.0,Only contains 40 energy shards and 10 dark ene...
28,213788,Ascension: Year Three Collector's Edition,SpaceAlien,7.0,Plus King Rat promo cards.
29,213788,Ascension: Year Three Collector's Edition,stormseeker75,7.0,I really don't like the Energy cards in here s...
34,213788,Ascension: Year Three Collector's Edition,Paletius,7.0,I didn't enjoy this one as much as the year tw...
43,213788,Ascension: Year Three Collector's Edition,Acward,7.0,We definitely enjoy owning this for the wonder...
45,213788,Ascension: Year Three Collector's Edition,hatbeard,7.0,Current Location: Brighton Solo: Yes
47,213788,Ascension: Year Three Collector's Edition,Chunkmonger,7.0,The art has improved since it's first release....
53,213788,Ascension: Year Three Collector's Edition,DurangoExile,7.0,Nice deck building game with very few rough ed...


In [71]:
comments_only.to_pickle('data_cleaned/master_comments_file.pkl')

In [72]:
del comments_only
gc.collect()

210

## Deploy - User and Item Ratings

In [73]:
raw_game_ratings = pd.read_pickle('data_dirty/raw_game_ratings.pkl')

In [74]:
ratings = raw_game_ratings[['BGGId', 'Rating', 'Username']]
ratings.head()

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP


In [75]:
del raw_game_ratings
gc.collect()

63

In [76]:
ratings.drop_duplicates(keep='first', inplace=True)

In [77]:
ratings.head()

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP


In [78]:
ratings.to_csv('data_kaggle/user_ratings.csv', index=False)

In [79]:
ratings.Username.nunique()

411375

In [80]:
game_ids_current = pd.read_pickle('data_cleaned/game_ids_current.pkl')
game_ids = list(game_ids_current)

In [81]:
user_ratings = {}

raw_ratings = {}

for item in game_ids:
    raw_ratings[item] = []

In [82]:
ratings.apply(lambda x: process_dataframe_ratings(x, user_ratings, raw_ratings), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
62415    None
62416    None
62417    None
62418    None
62419    None
Length: 18942215, dtype: object

In [83]:
user_ratings['Threnody']

{25190: 6.0,
 13: 6.0,
 3076: 6.0,
 822: 7.0,
 68448: 7.5,
 2651: 6.0,
 30549: 8.0,
 31260: 7.0,
 14996: 7.0,
 266192: 7.0,
 110327: 7.0,
 9209: 7.0,
 70323: 7.0,
 36218: 8.0,
 28143: 7.0,
 34635: 7.0,
 178900: 8.0,
 237182: 7.0,
 162886: 7.0,
 167791: 9.0,
 98778: 7.0,
 320: 4.0,
 201808: 7.0,
 41114: 7.0,
 230802: 8.0,
 124361: 7.5,
 18602: 7.0,
 157354: 7.5,
 169786: 9.0,
 148228: 8.0,
 65244: 7.0,
 1406: 3.0,
 157969: 7.0,
 170216: 8.0,
 146508: 6.5,
 37111: 8.0,
 129622: 8.0,
 224517: 8.0,
 50: 7.0,
 10547: 7.0,
 478: 7.5,
 11: 7.0,
 40834: 8.0,
 3955: 6.0,
 118: 6.0,
 199792: 8.0,
 291457: 8.0,
 27162: 7.0,
 50381: 6.0,
 2453: 7.0,
 192291: 7.5,
 3: 6.0,
 233867: 7.5,
 218603: 7.0,
 43443: 6.0,
 172818: 6.5,
 1294: 5.0,
 124708: 7.0,
 182028: 9.0,
 244521: 8.0,
 239188: 7.5,
 182874: 7.5,
 37046: 7.5,
 148949: 8.0,
 102652: 7.0,
 131357: 8.0,
 219513: 7.0,
 199561: 8.0,
 2921: 2.0,
 9216: 6.0,
 155426: 8.0,
 236457: 8.0,
 220877: 6.5,
 104006: 8.0,
 247763: 8.0,
 198994: 7.0,
 19

In [84]:
raw_ratings[213788]

[8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 7.8,
 7.6,
 7.50667,
 7.5,
 7.5,
 7.5,
 7.5,
 7.5,
 7.2,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 7.0,
 6.77778,
 6.5,
 6.5,
 6.5,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 3.0,
 1.5,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 9.02312,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 9.0,
 8.9,
 8.8,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.3,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,

In [ ]:
# save dictionary
with open('data_cleaned/game_raw_ratings.json', 'w') as convert_file:
    convert_file.write(json.dumps(raw_ratings))
    
with open('real_ratings/user_ratings_unscaled.json', 'w') as convert_file:
    convert_file.write(json.dumps(user_ratings))
    
del ratings
gc.collect() 

### Winnow Users to 5+ Ratings

In [ ]:
# Opening JSON file
with open('data_cleaned/user_ratings_unscaled.json') as json_file:
    user_ratings = json.load(json_file)

In [ ]:
all_users = list(user_ratings.keys())

In [ ]:
for user in all_users:

    if len(user_ratings[user]) < 5:
        print("Removing user "+user)
        del user_ratings[user]
        continue

In [ ]:
with open('data_cleaned/user_ratings_unscaled.json', 'w') as convert_file:
    convert_file.write(json.dumps(user_ratings))

## Ratings Distribution

In [85]:
# Load the storage dictionary for this block
with open('data_cleaned/game_raw_ratings.json') as json_file:
    raw_ratings = json.load(json_file) 

In [86]:
len(raw_ratings)

21925

In [87]:
ratings_distribution = pd.DataFrame()

for item in raw_ratings.keys():
    
    print(item)
    
    ratings_temp = pd.DataFrame(raw_ratings[item]).round(1)
    ratings_counts = pd.DataFrame(ratings_temp.value_counts()).sort_index().T
    
    ratings_distribution = ratings_distribution.append(ratings_counts)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
34
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
57
58
59
60
61
62
63
64
66
67
68
69
70
71
72
73
74
75
76
77
78
79
82
83
84
85
87
88
89
90
91
93
94
95
96
97
98
99
101
102
103
104
105
106
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
124
125
126
127
128
130
132
133
134
135
136
137
139
140
141
142
144
145
146
147
148
150
151
152
153
154
155
156
157
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
178
179
180
181
182
183
185
186
188
189
190
192
193
194
195
196
197
198
199
200
201
202
204
206
207
208
210
211
214
215
216
217
218
219
220
221
222
223
224
225
226
228
229
230
231
232
234
235
237
239
240
241
242
243
244
246
247
248
249
250
252
254
255
256
257
258
259
260
261
263
264
265
267
268
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
307
308
309
310
311
312
313
314
315


2574
2577
2578
2579
2580
2581
2582
2583
2584
2586
2589
2593
2596
2597
2598
2599
2600
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2616
2625
2626
2627
2629
2630
2633
2634
2635
2637
2638
2639
2640
2641
2643
2645
2649
2650
2651
2652
2653
2655
2656
2657
2664
2665
2666
2667
2668
2669
2672
2674
2676
2678
2679
2680
2681
2688
2689
2690
2693
2697
2698
2699
2701
2705
2706
2707
2709
2710
2713
2716
2717
2718
2719
2720
2721
2724
2725
2728
2730
2731
2733
2734
2736
2738
2740
2742
2743
2746
2748
2749
2750
2751
2753
2756
2757
2758
2759
2760
2762
2763
2764
2765
2766
2767
2770
2771
2773
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2788
2791
2792
2793
2794
2795
2799
2800
2802
2806
2808
2809
2811
2812
2814
2816
2817
2818
2819
2821
2823
2824
2826
2832
2833
2836
2838
2839
2840
2841
2842
2843
2844
2845
2853
2854
2855
2856
2857
2858
2860
2863
2864
2866
2867
2868
2871
2872
2875
2876
2878
2879
2881
2883
2884
2885
2887
2888
2889
2891
2895
2897
2901
2902
2903
2904
2907
2910
2911
2916
2917


5836
5837
5841
5843
5851
5854
5856
5858
5859
5860
5862
5863
5864
5867
5869
5870
5875
5876
5877
5878
5881
5882
5885
5886
5890
5893
5894
5895
5897
5899
5900
5902
5903
5905
5922
5923
5924
5926
5927
5928
5929
5930
5933
5934
5935
5936
5939
5940
5941
5942
5952
5953
5955
5956
5957
5958
5959
5960
5961
5964
5965
5966
5985
5988
5990
5991
5992
5993
5994
5996
5997
5998
5999
6000
6003
6004
6005
6006
6007
6008
6009
6010
6017
6020
6021
6022
6023
6024
6036
6037
6038
6039
6040
6041
6042
6044
6045
6046
6047
6049
6050
6052
6053
6054
6068
6085
6086
6087
6088
6090
6109
6112
6117
6118
6119
6121
6122
6124
6125
6129
6130
6132
6137
6146
6152
6153
6187
6191
6194
6197
6201
6202
6203
6205
6209
6210
6215
6219
6221
6226
6228
6230
6231
6232
6233
6235
6240
6245
6249
6250
6251
6253
6263
6266
6268
6270
6272
6281
6282
6283
6284
6333
6338
6339
6342
6346
6350
6351
6352
6353
6355
6360
6363
6364
6365
6366
6367
6368
6374
6375
6380
6381
6383
6386
6387
6388
6395
6397
6398
6399
6400
6402
6404
6405
6406
6411
6412
6414
6417
6420


10807
10808
10809
10811
10813
10814
10815
10816
10819
10820
10822
10829
10831
10836
10842
10845
10855
10869
10878
10880
10884
10885
10889
10890
10893
10898
10904
10906
10908
10912
10914
10915
10916
10917
10925
10934
10946
10947
10949
10960
10962
10964
10966
10968
10971
10973
10986
10989
10995
10996
10997
10998
10999
11001
11002
11011
11017
11027
11028
11030
11037
11038
11047
11050
11052
11054
11056
11057
11060
11071
11081
11083
11084
11085
11091
11093
11096
11104
11106
11110
11111
11112
11113
11115
11116
11121
11122
11124
11125
11126
11127
11129
11131
11132
11135
11136
11138
11139
11147
11149
11159
11160
11161
11166
11167
11168
11170
11177
11179
11182
11187
11188
11195
11200
11202
11208
11215
11229
11231
11235
11241
11248
11253
11258
11262
11265
11269
11274
11278
11283
11284
11287
11289
11292
11293
11320
11330
11339
11347
11351
11357
11370
11373
11385
11388
11393
11394
11396
11399
11402
11404
11407
11408
11412
11413
11414
11415
11416
11421
11428
11429
11432
11433
11435
11436
11437
1143

18902
18905
18907
18908
18913
18919
18931
18932
18945
18946
18961
18963
18966
18968
18975
18983
18985
19000
19001
19009
19027
19028
19030
19036
19039
19040
19044
19046
19048
19056
19062
19074
19081
19083
19100
19114
19115
19120
19127
19145
19161
19184
19186
19187
19188
19201
19212
19237
19246
19253
19257
19272
19274
19301
19304
19307
19329
19348
19349
19351
19358
19361
19363
19368
19370
19372
19377
19378
19379
19381
19392
19399
19409
19414
19416
19417
19418
19419
19425
19427
19432
19442
19447
19456
19464
19466
19475
19478
19484
19498
19505
19508
19512
19513
19526
19530
19531
19534
19537
19540
19544
19552
19554
19560
19571
19578
19591
19600
19614
19616
19621
19622
19623
19624
19630
19633
19634
19643
19646
19647
19650
19651
19653
19665
19671
19676
19679
19696
19697
19725
19727
19732
19735
19763
19764
19765
19766
19777
19793
19796
19802
19803
19811
19834
19841
19842
19854
19857
19876
19877
19878
19888
19902
19903
19904
19908
19909
19914
19915
19916
19918
19947
19948
19957
19969
19971
1997

...

207351
207370
207378
207398
207442
207486
207487
207568
207572
207602
207653
207670
207687
207691
207713
207719
207729
207753
207764
207783
207796
207809
207814
207830
207845
207848
207883
207898
207899
207905
207910
207911
207917
207919
207951
207965
207975
207987
207991
208023
208024
208073
208093
208134
208170
208217
208256
208316
208321
208323
208337
208340
208360
208377
208411
208419
208428
208480
208533
208543
208583
208661
208663
208670
208700
208736
208754
208766
208773
208775
208800
208801
208804
208807
208808
208887
208888
208895
208918
208995
209001
209003
209010
209014
209033
209053
209090
209095
209113
209136
209166
209168
209190
209206
209220
209222
209225
209280
209282
209283
209284
209289
209290
209320
209322
209324
209325
209344
209366
209378
209383
209418
209423
209447
209450
209452
209488
209506
209511
209530
209538
209541
209542
209551
209567
209578
209641
209642
209644
209649
209653
209660
209661
209664
209667
209671
209672
209685
209696
209714
209716
209734
209737

231273
231280
231302
231309
231315
231316
231327
231331
231362
231367
231368
231392
231450
231457
231476
231477
231484
231538
231553
231554
231559
231567
231573
231574
231575
231581
231587
231611
231618
231620
231624
231639
231644
231664
231665
231666
231674
231683
231696
231733
231748
231801
231812
231819
231824
231837
231854
231872
231876
231878
231920
231939
231942
231954
231960
231962
231966
231972
231991
231999
232041
232043
232045
232078
232079
232090
232119
232134
232139
232162
232166
232201
232210
232216
232219
232238
232242
232250
232254
232267
232300
232303
232318
232345
232348
232351
232353
232358
232361
232385
232403
232405
232412
232414
232417
232420
232437
232446
232473
232478
232481
232509
232510
232520
232524
232595
232661
232666
232732
232800
232822
232823
232824
232827
232829
232830
232831
232832
232874
232894
232895
232900
232918
232924
232944
232950
232956
232974
232979
232980
232981
232988
233006
233015
233016
233018
233020
233033
233048
233070
233078
233080
233085

253506
253512
253564
253568
253574
253607
253608
253618
253635
253652
253664
253679
253684
253696
253705
253719
253743
253756
253763
253766
253786
253807
253819
253861
253862
254017
254018
254025
254029
254088
254110
254123
254132
254135
254138
254145
254149
254157
254178
254188
254189
254192
254193
254195
254213
254226
254227
254230
254239
254322
254341
254386
254394
254399
254415
254417
254419
254427
254494
254498
254513
254532
254546
254585
254588
254591
254613
254617
254618
254619
254634
254639
254640
254681
254683
254686
254695
254708
254713
254743
254759
254846
254888
254894
254923
254936
254938
254976
254992
254994
255013
255020
255027
255029
255034
255037
255042
255165
255171
255175
255227
255235
255249
255255
255262
255272
255291
255293
255310
255332
255335
255356
255360
255363
255381
255392
255393
255396
255455
255456
255478
255484
255507
255516
255524
255570
255586
255594
255597
255608
255615
255618
255633
255635
255639
255640
255642
255651
255653
255658
255659
255664
255668

276622
276633
276641
276654
276669
276760
276779
276801
276817
276830
276832
276856
276880
276894
276923
276924
276925
276997
277003
277017
277018
277030
277035
277061
277080
277085
277109
277131
277184
277405
277410
277424
277434
277448
277458
277469
277475
277481
277538
277565
277597
277611
277629
277659
277662
277671
277672
277699
277700
277779
277892
277901
277902
277903
277927
277930
277931
277980
278042
278120
278139
278142
278218
278292
278297
278304
278305
278322
278359
278373
278402
278413
278484
278553
278554
278693
278751
278783
278824
278971
278989
279014
279056
279087
279135
279198
279204
279254
279306
279307
279328
279348
279419
279497
279527
279537
279556
279581
279613
279643
279644
279646
279649
279720
279727
279741
279776
279869
279884
279900
279911
279928
279931
279967
279975
279996
280032
280041
280106
280107
280131
280132
280136
280143
280148
280149
280162
280201
280202
280203
280281
280282
280304
280378
280430
280453
280475
280480
280501
280566
280575
280655
280729

307191
307305
307369
307386
307621
307656
307715
307830
307832
307844
307862
307963
307971
307997
308028
308119
308126
308329
308354
308357
308368
308385
308386
308388
308416
308493
308500
308529
308532
308565
308566
308621
308652
308755
308762
308765
308870
308948
308970
308989
309000
309003
309081
309105
309109
309110
309113
309129
309207
309250
309297
309319
309341
309408
309430
309545
309600
309630
309752
309857
309862
309917
310031
310076
310100
310192
310194
310196
310198
310203
310363
310408
310442
310448
310578
310610
310611
310612
310632
310641
310726
310779
310789
310846
310873
310880
310885
310888
310953
310968
311004
311020
311191
311192
311193
311209
311322
311330
311464
311465
311659
311686
311702
311708
311715
311751
311828
311829
311894
311918
311990
312059
312182
312251
312259
312267
312289
312318
312346
312372
312463
312484
312594
312613
312618
312626
312726
312738
312744
312762
312767
312786
312804
312904
312951
312965
312968
313000
313002
313008
313010
313085
313090

In [88]:
#ratings_distribution = pd.read_pickle('data_cleaned/ratings_distribution.pkl')

In [89]:
ratings_distribution.set_axis(list(raw_ratings.keys()), axis=0, inplace=True)

In [90]:
ratings_distribution.head()

,0.0,0.1,0.5,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6.0,6.1,6.2,6.3,6.4,6.5,6.6,6.7,6.8,6.9,7.0,7.1,7.2,7.3,7.4,7.5,7.6,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6,8.7,8.8,8.9,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,53.0,1.0,NaN,NaN,1.0,5.0,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,212.0,NaN,NaN,NaN,NaN,15.0,NaN,1.0,1.0,2.0,513.0,3.0,4.0,2.0,6.0,76.0,NaN,6.0,11.0,7.0,986.0,3.0,19.0,4.0,4.0,229.0,7.0,9.0,32.0,19.0,1265.0,6.0,34.0,9.0,10.0,225.0,12.0,10.0,19.0,8.0,843.0,4.0,11.0,5.0,11.0,86.0,3.0,4.0,6.0,8.0,426.0
2,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,65.0,1.0,NaN,NaN,NaN,8.0,NaN,NaN,1.0,NaN,140.0,2.0,NaN,NaN,NaN,14.0,2.0,2.0,2.0,NaN,142.0,1.0,1.0,NaN,NaN,10.0,NaN,2.0,NaN,1.0,80.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,32.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,17.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,150.0,NaN,2.0,NaN,1.0,12.0,NaN,1.0,4.0,NaN,454.0,1.0,2.0,2.0,NaN,36.0,1.0,1.0,8.0,1.0,1564.0,2.0,5.0,7.0,8.0,367.0,7.0,13.0,46.0,20.0,3995.0,20.0,83.0,55.0,47.0,1050.0,42.0,51.0,127.0,35.0,4091.0,40.0,56.0,23.0,22.0,486.0,28.0,13.0,33.0,15.0,1497.0,11.0,20.0,7.0,4.0,77.0,3.0,1.0,5.0,3.0,477.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,39.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,88.0,NaN,NaN,NaN,NaN,19.0,NaN,NaN,1.0,1.0,94.0,NaN,2.0,NaN,1.0,9.0,NaN,NaN,1.0,NaN,46.0,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,1.0,2.0,1.0,17.0,NaN,NaN,NaN,NaN,819.0,1.0,5.0,2.0,4.0,82.0,5.0,7.0,17.0,3.0,2352.0,7.0,18.0,12.0,12.0,483.0,11.0,13.0,59.0,29.0,5039.0,28.0,98.0,53.0,59.0,1065.0,46.0,46.0,118.0,35.0,4300.0,24.0,64.0,21.0,25.0,446.0,14.0,17.0,32.0,9.0,1792.0,11.0,18.0,8.0,4.0,82.0,7.0,9.0,10.0,5.0,905.0


In [91]:
ratings_distribution.fillna(0, inplace=True)

In [92]:
ratings_distribution.head()

,0.0,0.1,0.5,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6.0,6.1,6.2,6.3,6.4,6.5,6.6,6.7,6.8,6.9,7.0,7.1,7.2,7.3,7.4,7.5,7.6,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6,8.7,8.8,8.9,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,53.0,1.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,212.0,0.0,0.0,0.0,0.0,15.0,0.0,1.0,1.0,2.0,513.0,3.0,4.0,2.0,6.0,76.0,0.0,6.0,11.0,7.0,986.0,3.0,19.0,4.0,4.0,229.0,7.0,9.0,32.0,19.0,1265.0,6.0,34.0,9.0,10.0,225.0,12.0,10.0,19.0,8.0,843.0,4.0,11.0,5.0,11.0,86.0,3.0,4.0,6.0,8.0,426.0
2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,65.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0,1.0,0.0,140.0,2.0,0.0,0.0,0.0,14.0,2.0,2.0,2.0,0.0,142.0,1.0,1.0,0.0,0.0,10.0,0.0,2.0,0.0,1.0,80.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,32.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,17.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,150.0,0.0,2.0,0.0,1.0,12.0,0.0,1.0,4.0,0.0,454.0,1.0,2.0,2.0,0.0,36.0,1.0,1.0,8.0,1.0,1564.0,2.0,5.0,7.0,8.0,367.0,7.0,13.0,46.0,20.0,3995.0,20.0,83.0,55.0,47.0,1050.0,42.0,51.0,127.0,35.0,4091.0,40.0,56.0,23.0,22.0,486.0,28.0,13.0,33.0,15.0,1497.0,11.0,20.0,7.0,4.0,77.0,3.0,1.0,5.0,3.0,477.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,88.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,1.0,1.0,94.0,0.0,2.0,0.0,1.0,9.0,0.0,0.0,1.0,0.0,46.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,1.0,2.0,1.0,17.0,0.0,0.0,0.0,0.0,819.0,1.0,5.0,2.0,4.0,82.0,5.0,7.0,17.0,3.0,2352.0,7.0,18.0,12.0,12.0,483.0,11.0,13.0,59.0,29.0,5039.0,28.0,98.0,53.0,59.0,1065.0,46.0,46.0,118.0,35.0,4300.0,24.0,64.0,21.0,25.0,446.0,14.0,17.0,32.0,9.0,1792.0,11.0,18.0,8.0,4.0,82.0,7.0,9.0,10.0,5.0,905.0


In [93]:
ratings_distribution['total_ratings'] = ratings_distribution.sum(axis=1)

In [94]:
#ratings_distribution = ratings_distribution.T.reset_index().T

In [95]:
ratings_distribution.reset_index(inplace=True)

In [96]:
ratings_distribution.head()

,index,0.0,0.1,0.5,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6.0,6.1,6.2,6.3,6.4,6.5,6.6,6.7,6.8,6.9,7.0,7.1,7.2,7.3,7.4,7.5,7.6,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6,8.7,8.8,8.9,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0,total_ratings
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,53.0,1.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,212.0,0.0,0.0,0.0,0.0,15.0,0.0,1.0,1.0,2.0,513.0,3.0,4.0,2.0,6.0,76.0,0.0,6.0,11.0,7.0,986.0,3.0,19.0,4.0,4.0,229.0,7.0,9.0,32.0,19.0,1265.0,6.0,34.0,9.0,10.0,225.0,12.0,10.0,19.0,8.0,843.0,4.0,11.0,5.0,11.0,86.0,3.0,4.0,6.0,8.0,426.0,5352.0
1,2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,65.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0,1.0,0.0,140.0,2.0,0.0,0.0,0.0,14.0,2.0,2.0,2.0,0.0,142.0,1.0,1.0,0.0,0.0,10.0,0.0,2.0,0.0,1.0,80.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,32.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,17.0,562.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,150.0,0.0,2.0,0.0,1.0,12.0,0.0,1.0,4.0,0.0,454.0,1.0,2.0,2.0,0.0,36.0,1.0,1.0,8.0,1.0,1564.0,2.0,5.0,7.0,8.0,367.0,7.0,13.0,46.0,20.0,3995.0,20.0,83.0,55.0,47.0,1050.0,42.0,51.0,127.0,35.0,4091.0,40.0,56.0,23.0,22.0,486.0,28.0,13.0,33.0,15.0,1497.0,11.0,20.0,7.0,4.0,77.0,3.0,1.0,5.0,3.0,477.0,15148.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,88.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,1.0,1.0,94.0,0.0,2.0,0.0,1.0,9.0,0.0,0.0,1.0,0.0,46.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,342.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,1.0,2.0,1.0,17.0,0.0,0.0,0.0,0.0,819.0,1.0,5.0,2.0,4.0,82.0,5.0,7.0,17.0,3.0,2352.0,7.0,18.0,12.0,12.0,483.0,11.0,13.0,59.0,29.0,5039.0,28.0,98.0,53.0,59.0,1065.0,46.0,46.0,118.0,35.0,4300.0,24.0,64.0,21.0,25.0,446.0,14.0,17.0,32.0,9.0,1792.0,11.0,18.0,8.0,4.0,82.0,7.0,9.0,10.0,5.0,905.0,18387.0


In [97]:
ratings_distribution.rename(columns={'index':'BGGId'}, inplace=True)

In [98]:
ratings_distribution['BGGId'] = ratings_distribution['BGGId'].astype('int64')

In [99]:
ratings_distribution.head()

,BGGId,0.0,0.1,0.5,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6.0,6.1,6.2,6.3,6.4,6.5,6.6,6.7,6.8,6.9,7.0,7.1,7.2,7.3,7.4,7.5,7.6,7.7,7.8,7.9,8.0,8.1,8.2,8.3,8.4,8.5,8.6,8.7,8.8,8.9,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0,total_ratings
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,53.0,1.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,212.0,0.0,0.0,0.0,0.0,15.0,0.0,1.0,1.0,2.0,513.0,3.0,4.0,2.0,6.0,76.0,0.0,6.0,11.0,7.0,986.0,3.0,19.0,4.0,4.0,229.0,7.0,9.0,32.0,19.0,1265.0,6.0,34.0,9.0,10.0,225.0,12.0,10.0,19.0,8.0,843.0,4.0,11.0,5.0,11.0,86.0,3.0,4.0,6.0,8.0,426.0,5352.0
1,2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,65.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0,1.0,0.0,140.0,2.0,0.0,0.0,0.0,14.0,2.0,2.0,2.0,0.0,142.0,1.0,1.0,0.0,0.0,10.0,0.0,2.0,0.0,1.0,80.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,32.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,17.0,562.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,150.0,0.0,2.0,0.0,1.0,12.0,0.0,1.0,4.0,0.0,454.0,1.0,2.0,2.0,0.0,36.0,1.0,1.0,8.0,1.0,1564.0,2.0,5.0,7.0,8.0,367.0,7.0,13.0,46.0,20.0,3995.0,20.0,83.0,55.0,47.0,1050.0,42.0,51.0,127.0,35.0,4091.0,40.0,56.0,23.0,22.0,486.0,28.0,13.0,33.0,15.0,1497.0,11.0,20.0,7.0,4.0,77.0,3.0,1.0,5.0,3.0,477.0,15148.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,88.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,1.0,1.0,94.0,0.0,2.0,0.0,1.0,9.0,0.0,0.0,1.0,0.0,46.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,342.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,1.0,2.0,1.0,17.0,0.0,0.0,0.0,0.0,819.0,1.0,5.0,2.0,4.0,82.0,5.0,7.0,17.0,3.0,2352.0,7.0,18.0,12.0,12.0,483.0,11.0,13.0,59.0,29.0,5039.0,28.0,98.0,53.0,59.0,1065.0,46.0,46.0,118.0,35.0,4300.0,24.0,64.0,21.0,25.0,446.0,14.0,17.0,32.0,9.0,1792.0,11.0,18.0,8.0,4.0,82.0,7.0,9.0,10.0,5.0,905.0,18387.0


In [ ]:
ratings_distribution.to_pickle('data_cleaned/ratings_distribution.pkl')

In [100]:
ratings_distribution.to_csv('data_kaggle/ratings_distribution.csv', index=False)